# Task 3: Pre-trained transformers

### Aim
In this homework, the aim of the Argilla's dataset is the following:

- Predicting the medical field from which a medical note comes from.

To solve this classification problem, we will use three types of algorithm:

1) Machine learning algorithm: Support vector machine and Logistic regression
2) Transformer-encoder: Medical Bert model + classification head
3) Transformer-decoder: Qwen 2.5

### Accuracy:

Accuracy is tested, as instructed, using:
- Macro-f1 metric
- Human visual inspection

## 1) Machine learning

In [1]:
import numpy as np
import sklearn
import matplotlib
import transformers
import pandas as pd
import tqdm
import torch
import spacy
import nltk
import evaluate

/Users/matthias/miniconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.1 Dataset import

We re-use code from task 1 to import our argilla dataset, where we will only keep the text and the labels.

In [2]:
import sys, site
print(sys.executable)
print("USER_SITE:", site.getusersitepackages())
print("sys.path[0:5]:", sys.path[:5])

/Users/matthias/miniconda3/envs/nlp/bin/python
USER_SITE: /Users/matthias/.local/lib/python3.11/site-packages
sys.path[0:5]: ['/Users/matthias/miniconda3/envs/nlp/lib/python311.zip', '/Users/matthias/miniconda3/envs/nlp/lib/python3.11', '/Users/matthias/miniconda3/envs/nlp/lib/python3.11/lib-dynload', '', '/Users/matthias/miniconda3/envs/nlp/lib/python3.11/site-packages']


In [3]:

pd.set_option('display.max_colwidth', 200)

df = pd.read_parquet("hf://datasets/argilla/medical-domain/data/train-00000-of-00001-67e4e7207342a623.parquet")

def extract_label(pred):
    if isinstance(pred, (list, np.ndarray)) and len(pred) > 0 and isinstance(pred[0], dict):
        return pred[0].get("label")
    return None

df['label'] = df['prediction'].apply(extract_label)
df['text_length'] = df['metrics'].apply(lambda x: x.get('text_length') if isinstance(x, dict) else None)

# drop empty columns
df = df.drop(columns=['inputs', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'multi_label', 'explanation', 'metadata', 'status', 'event_timestamp', 'metrics'], errors='ignore')

#print(df.head)
df.head()

,text,id,label,text_length
0,"PREOPERATIVE DIAGNOSIS:, Iron deficiency anemia.,POSTOPERATIVE DIAGNOSIS:, Diverticulosis.,PROCEDURE:, Colonoscopy.,MEDICATIONS: , MAC.,PROCEDURE: , The Olympus pediatric variable colonoscope w...",00001265-03e2-47b2-b6cf-bed32dad2fa9,Gastroenterology,1085
1,"CLINICAL INDICATION: ,Normal stress test.,PROCEDURES PERFORMED:,1. Left heart cath.,2. Selective coronary angiography.,3. LV gram.,4. Right femoral arteriogram.,5. Mynx closure device.,PROCE...",0007edf0-1413-4b16-8212-3a13c2ab4e43,Surgery,1798
2,"FINDINGS:,Axial scans were performed from L1 to S2 and reformatted images were obtained in the sagittal and coronal planes.,Preliminary scout film demonstrates anterior end plate spondylosis at T1...",00097d1e-1357-4447-a39a-fe8f8b7c36ae,Radiology,1141
3,"PREOPERATIVE DIAGNOSIS: , Blood loss anemia.,POSTOPERATIVE DIAGNOSES:,1. Diverticulosis coli.,2. Internal hemorrhoids.,3. Poor prep.,PROCEDURE PERFORMED:, Colonoscopy with photos.,ANESTHESIA: ...",001622b6-0182-4fee-9881-ae15e81ce836,Surgery,1767
4,"REASON FOR VISIT: ,Elevated PSA with nocturia and occasional daytime frequency.,HISTORY: , A 68-year-old male with a history of frequency and some outlet obstructive issues along with irritative ...",0029245f-8b45-4796-ba09-7760612289c6,SOAP / Chart / Progress Notes,1519


#### 1.2 Data set formatting

The aim of this section is to:
- Split the data set in training and testing set.
- Transforming a specific word "x" in the product of :
- _Its frequency in a specific document d (TF)_
- _Total number of documents/ number of times "x" appears per document_ (IDF)

In [4]:
###################################
#0. Split data set into train/test
#################################

# This code is inspired from : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

from sklearn.model_selection import train_test_split
X=df["text"]
y=df["label"]

# We create two data sets by splitting:
# - 80% of data set in training set
# - 20% in test set
# - Applying stratify=y to make sure that categories are roughly equal with train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y) # I split the text : 80% training, 20% test

############################
# 1. TF-IDF
############################

# Using sklearn TfidfVectorizer, we can directly pre-processed our text:
# - everything in lowercase
# - tokenize words
# - every feature of same length

# We finally return for each document d and word k: k frequency in document d * inverse frequency of k in all documents (TF-IDF).

## This code is adapted from https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(strip_accents="unicode", # I want to strip all accents
                             lowercase=True,  # I want everything lowercase
                             stop_words="english", # We delete English-stop words
                             min_df=5,  # I want words to be at least in 5 documents
                             max_df=0.8, # very frequent words are not useful to distinguish between documents
                             ngram_range=(1,2) # include also pairs of words
                             )


X_train_TDIFD = vectorizer.fit_transform(X_train)
X_test_TDIFD=vectorizer.transform(X_test) # I transform X_test according to our TfidVectorizer

### 1.3 Linear SVM

In [5]:
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

SVM=LinearSVC(random_state=0, tol=1e-5,class_weight="balanced")
SVM.fit(X_train_TDIFD,y_train)

SVM.score(X_test_TDIFD,y_test) # Accuracy

f1_score_macro_SVM=f1_score(y_test, SVM.predict(X_test_TDIFD), average='macro') # Macro F_1 score -->"harmonic mean of the precision and recall" https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
print("F1 score macro SVM: ",f1_score_macro_SVM)

F1 score macro SVM:  0.21555928764956586


F1 score macro SVM:  0.21555928764956586

In [6]:
for i in range(3):
    print("Predicted class:",SVM.predict(X_test_TDIFD[i]))
    print("Text: ", X_test.iloc[i][0:500]) ### 500 hundred first characters

Predicted class: [' Cardiovascular / Pulmonary']
Text:  PREOPERATIVE DIAGNOSIS: , Coronary artery disease.,POSTOPERATIVE DIAGNOSIS: , Coronary artery disease plus intimal calcification in the mid abdominal aorta without significant stenosis.,DESCRIPTION OF PROCEDURE:,LEFT HEART CATHETERIZATION WITH ANGIOGRAPHY AND MID ABDOMINAL AORTOGRAPHY:,Under local anesthesia with 2% lidocaine with premedication, a right groin preparation was done.  Using the percutaneous Seldinger technique via the right femoral artery, a left heart catheterization was performed
Predicted class: [' Nephrology']
Text:  EXAM: , CT of abdomen with and without contrast.  CT-guided needle placement biopsy.,HISTORY: , Left renal mass.,TECHNIQUE: , Pre and postcontrast enhanced images were acquired through the kidneys.,FINDINGS: , Comparison made to the prior MRI.  There is re-demonstration of multiple bilateral cystic renal lesions.  Several of these demonstrate high attenuation in the precontrast phase of the exam sugg

### 1.4 Logistic regression

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

lr = LogisticRegression(
    random_state=0,
    tol=1e-5,
    class_weight="balanced",
    solver="saga",
    max_iter=3000,
    n_jobs=-1
)

param_grid = {"C": [0.1, 1.0, 10.0]}

grid = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train_TDIFD, y_train)

LR = grid.best_estimator_
print("Best C:", grid.best_params_["C"])

y_pred = LR.predict(X_test_TDIFD)
print("F1 score macro LR (tuned):", f1_score(y_test, y_pred, average="macro"))
print("Accuracy (tuned):", LR.score(X_test_TDIFD, y_test))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/Users/matthias/miniconda3/envs/nlp/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Users/matthias/miniconda3/envs/nlp/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Users/matthias/miniconda3/envs/nlp/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Users/matthias/miniconda3/envs/nlp/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 a

Best C: 0.1
F1 score macro LR (tuned): 0.3734146663463421
Accuracy (tuned): 0.3319919517102616


Best C: 0.1 \
F1 score macro LR (tuned): 0.3734146663463421 \
Accuracy (tuned): 0.3319919517102616 

In [8]:
for i in range(3):
    print("Predicted class:",LR.predict(X_test_TDIFD[i]))
    print("Text: ", X_test.iloc[i][0:500]) ### 500 hundred first characters

Predicted class: [' Cardiovascular / Pulmonary']
Text:  PREOPERATIVE DIAGNOSIS: , Coronary artery disease.,POSTOPERATIVE DIAGNOSIS: , Coronary artery disease plus intimal calcification in the mid abdominal aorta without significant stenosis.,DESCRIPTION OF PROCEDURE:,LEFT HEART CATHETERIZATION WITH ANGIOGRAPHY AND MID ABDOMINAL AORTOGRAPHY:,Under local anesthesia with 2% lidocaine with premedication, a right groin preparation was done.  Using the percutaneous Seldinger technique via the right femoral artery, a left heart catheterization was performed
Predicted class: [' Nephrology']
Text:  EXAM: , CT of abdomen with and without contrast.  CT-guided needle placement biopsy.,HISTORY: , Left renal mass.,TECHNIQUE: , Pre and postcontrast enhanced images were acquired through the kidneys.,FINDINGS: , Comparison made to the prior MRI.  There is re-demonstration of multiple bilateral cystic renal lesions.  Several of these demonstrate high attenuation in the precontrast phase of the exam sugg

##### _N.B: XGBoost_

Considering the high dimensionality of our data , XGboost takes too much time to run and SVM or LR are already strong baseline ML algorithm to compare our transformers to.

#### Conclusion

We select the logistic regression as our machine learning classifier, with an F1 macro score of **0.37**

The "human eye" test is convincing, with the algorithm giving meaningful labels to medical notes.

## 2. Encoder task

#### Model specification

We decided to use MedBERT Model. This is an encoder transformer, pre-trained for NER.

We will add a classification head to it (using AutoModelForSequenceClassification) in order to use it for prediction. This head has  untrained weights.

This model was used because of its pre-training on medical data, notably on  "a collection of clinical notes released in N2C2 2018 and N2C2 2022 challenges", which are highly relevant for our task.

#### 2.1 Model import

In [6]:
import sys, torch
print("python:", sys.executable)
print("torch version:", torch.__version__)
print("torch file:", torch.__file__)

python: /usr/bin/python3
torch version: 2.9.0+cu126
torch file: /usr/local/lib/python3.12/dist-packages/torch/__init__.py


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, set_seed

import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"  # disables HF progress bars (fixes progress updater crashes)

set_seed(42)

tokenizer = AutoTokenizer.from_pretrained("Charangan/MedBERT")

model = AutoModelForSequenceClassification.from_pretrained("Charangan/MedBERT",num_labels=40, resume_download=True)

# This code is adapted from https://huggingface.co/transformers/v4.2.2/training.html?

# I am freezing the encoder, but allowing to update weights of the classification head
for param in model.base_model.parameters():
    param.requires_grad = False

tokenizer_config.json:   0%|          | 0.00/417 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Charangan/MedBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### dataset formatting

In [8]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

dataset = dataset.select_columns(['text', 'label'])

# Labels are string, I need to change them as numbers.
labels=dataset.unique("label")

# I create a dictionnary that take label as key and return a value.


label2id = {
    "Gastroenterology": 0,
    "Surgery": 1,
    "Radiology": 2,
    "SOAP / Chart / Progress Notes": 3,
    "Letters": 4,
    "Lab Medicine - Pathology": 5,
    "Consult - History and Phy.": 6,
    "Podiatry": 7,
    "General Medicine": 8,
    "Psychiatry / Psychology": 9,
    "Cardiovascular / Pulmonary": 10,
    "Urology": 11,
    "Ophthalmology": 12,
    "Physical Medicine - Rehab": 13,
    "Neurology": 14,
    "Autopsy": 15,
    "Orthopedic": 16,
    "Hematology - Oncology": 17,
    "Allergy / Immunology": 18,
    "Pediatrics - Neonatal": 19,
    "Dentistry": 20,
    "Neurosurgery": 21,
    "Pain Management": 22,
    "Nephrology": 23,
    "Emergency Room Reports": 24,
    "Obstetrics / Gynecology": 25,
    "Speech - Language": 26,
    "Diets and Nutritions": 27,
    "Endocrinology": 28,
    "IME-QME-Work Comp etc.": 29,
    "Cosmetic / Plastic Surgery": 30,
    "Discharge Summary": 31,
    "ENT - Otolaryngology": 32,
    "Chiropractic": 33,
    "Office Notes": 34,
    "Dermatology": 35,
    "Sleep Medicine": 36,
    "Rheumatology": 37,
    "Hospice - Palliative Care": 38,
    "Bariatrics": 39,
}

# function for matching key to values
# Map will gives me one row of my dataset, into a dictionnary form.
# So i want to :
# 1) extract label value from dictionnary
# 2) replace it using my dictionnary with a numerical value
def matching(example):
    label=example["label"].strip() # labels have a whitespace as first character, that i strip
    example["label"]=label2id[label]
    return example

dataset=dataset.map(matching)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Map:   0%|          | 0/4966 [00:00<?, ? examples/s]

In [9]:
from datasets import load_dataset
from datasets import ClassLabel
from datasets import DatasetDict


# We firstly rename our column  from label to labels to match trainer wrappers
dataset = dataset.rename_column("label", "labels")
# dataset = dataset.cast_column("labels", ClassLabel(num_classes=40))
id2label = {v: k for k, v in label2id.items()}
dataset = dataset.cast_column("labels", ClassLabel(names=[id2label[i] for i in range(40)]))

## train/test=80/20
splits = dataset.train_test_split(
    test_size=0.2,
    stratify_by_column="labels", # We stratify once again to have the same
    seed=42                      # rate of classes between to data frames
)

# split train into train/validation
train_val = splits["train"].train_test_split(
    test_size=0.125,   # 12.5% of 80% = 10% overall
    stratify_by_column="labels",
    seed=42
)

# single DatasetDict with 3 splits 70/10/20=train/val/test
final_df = DatasetDict({
    "train": train_val["train"],
    "validation": train_val["test"],
    "test": splits["test"]
})

### Now, we need to tokenize our data set. Adapted from: https://huggingface.co/docs/datasets/use_dataset

def tokenization(example):
    return tokenizer(example["text"], truncation=True, max_length=512) # i will truncate every exmaple that are longer than 512  token. This is
                                                                       # the max input size of our model

final_df_tokenized = final_df.map(tokenization, batched=True)


final_df_tokenized.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

Casting the dataset:   0%|          | 0/4966 [00:00<?, ? examples/s]

Map:   0%|          | 0/3475 [00:00<?, ? examples/s]

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Map:   0%|          | 0/994 [00:00<?, ? examples/s]

In [10]:
print("Train, validation and test df dimensions:")
print(final_df)

Train, validation and test df dimensions:
DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 3475
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 497
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 994
    })
})


#### Define testing metrics (accuracy, f1 macro)

In [11]:
# This code is adapted from :
# https://discuss.huggingface.co/t/how-to-get-the-predicted-labels-per-epoch-or-step-for-the-huggingface-transformers-trainer/12078/2?
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#### training arguments

In [ ]:
# This code is adapted from : https://huggingface.co/transformers/v4.2.2/training.html?
from transformers import TrainingArguments
from transformers import set_seed

set_seed(42)

training_args = TrainingArguments(
    output_dir='.',                  # output directory
    num_train_epochs=3,              # total # of training epochs --> small, as we only train the head
    per_device_train_batch_size=8,   # batch size per device during training --> small, as i run that on limited GPU time (Google collab set-up)
    per_device_eval_batch_size=16,   # batch size for evaluation --> small, as i run that on limited GPU time (Google collab set-up)
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    push_to_hub=False,
    fp16=True,
    data_seed=42,
    seed=42,
    report_to="none",   # avoids wandb prompt
)


#### training loop

In [ ]:
# This code is adapted from : https://huggingface.co/transformers/v4.2.2/training.html?
from transformers import Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=final_df_tokenized["train"],         # training dataset
    eval_dataset= final_df_tokenized["validation"],          # evaluation dataset
    data_collator=data_collator, # allows dynamical padding --> every batch will have the same lenghts, which is max_length of this batch
    compute_metrics=compute_metrics # added to return f1
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss
500,3.133900
1000,2.850400


TrainOutput(global_step=1305, training_loss=2.963610699533046, metrics={'train_runtime': 349.9408, 'train_samples_per_second': 29.791, 'train_steps_per_second': 3.729, 'total_flos': 2743868604211200.0, 'train_loss': 2.963610699533046, 'epoch': 3.0})

#### Evaluate accuracy

In [ ]:
trainer.evaluate(eval_dataset=final_df_tokenized["test"])

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 2.837169885635376,
 'eval_accuracy': 0.2625754527162978,
 'eval_f1': 0.01979835050585184,
 'eval_precision': 0.03741178912411789,
 'eval_recall': 0.03678243082143638,
 'eval_runtime': 30.5963,
 'eval_samples_per_second': 32.488,
 'eval_steps_per_second': 2.059,
 'epoch': 3.0}

{'eval_loss': 2.837169885635376, \
 'eval_accuracy': 0.2625754527162978,\
 'eval_f1': 0.01979835050585184,\
 'eval_precision': 0.03741178912411789,\
 'eval_recall': 0.03678243082143638,\
 'eval_runtime': 30.5963,\
 'eval_samples_per_second': 32.488,\
 'eval_steps_per_second': 2.059,\
 'epoch': 3.0}

for i in range(3):
    print("Predicted class:",LR.predict(X_test_TDIFD[i]))
    print("Text: ", X_test.iloc[i][0:500]) ### 500 hundred first characters

### Human check

In [ ]:
output=trainer.predict(final_df_tokenized["test"])

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
id2label = {v: k for k, v in label2id.items()}

In [ ]:
import random
import numpy as np
import pandas as pd

def human_check_from_output(
    output,
    raw_test,
    id2label,
    n=20,
    seed=42,
    text_chars=500
):
    random.seed(seed)

    pred_ids = np.argmax(output.predictions, axis=1)

    idxs = random.sample(range(len(raw_test)), k=min(n, len(raw_test)))

    rows = []
    for i in idxs:
        gold_id = raw_test[i]["labels"]
        pred_id = int(pred_ids[i])

        rows.append({
            "idx": i,
            "correct": gold_id == pred_id,
            "gold_label": id2label[gold_id],
            "pred_label": id2label[pred_id],
            "text": raw_test[i]["text"][:text_chars].replace("\n", " ")
        })

    df = pd.DataFrame(rows).sort_values("correct")
    print(f"Human check | n={len(df)} | sample accuracy = {df['correct'].mean():.2%}")
    return df


In [ ]:
hc_encoder_df = human_check_from_output(
    output=output,
    raw_test=final_df["test"],
    id2label=id2label,
    n=25,
    seed=42
)

hc_encoder_df


Human check | n=25 | sample accuracy = 8.00%


,idx,correct,gold_label,pred_label,text
0,654,False,Gastroenterology,Surgery,"The patient's abdomen was prepped and draped in the usual sterile fashion. A subumbilical skin incision was made. The Veress needle was inserted, and the patient's abdominal cavity was insufflat..."
21,238,False,Discharge Summary,Consult - History and Phy.,"CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis..."
20,223,False,Orthopedic,Neurology,Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature. Motor strength was 5/5 on the MRC scale...
19,95,False,Pain Management,Orthopedic,"COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:, After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form. The patient was pl..."
18,30,False,Neurosurgery,Surgery,"PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:, MAC (monitored ane..."
17,32,False,Surgery,Gastroenterology,"INDICATION: , Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE: ,"
16,432,False,General Medicine,Consult - History and Phy.,"CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis. She ha..."
15,604,False,Ophthalmology,Consult - History and Phy.,Her past medical history includes insulin requiring diabetes mellitus for the past 28 years. She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease. ...
14,89,False,Consult - History and Phy.,Nephrology,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
13,558,False,Cardiovascular / Pulmonary,Surgery,"PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc..."


Human check | n=25 | sample accuracy = 8.00%

<div id="df-0ee37ca0-1b4a-44f9-8d06-c8ae1a34dec3" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold_label</th>
      <th>pred_label</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>Surgery</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>21</th>
      <td>238</td>
      <td>False</td>
      <td>Discharge Summary</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis...</td>
    </tr>
    <tr>
      <th>20</th>
      <td>223</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Neurology</td>
      <td>Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature.  Motor strength was 5/5 on the MRC scale...</td>
    </tr>
    <tr>
      <th>19</th>
      <td>95</td>
      <td>False</td>
      <td>Pain Management</td>
      <td>Orthopedic</td>
      <td>COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:,  After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form.  The patient was pl...</td>
    </tr>
    <tr>
      <th>18</th>
      <td>30</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:,  MAC (monitored ane...</td>
    </tr>
    <tr>
      <th>17</th>
      <td>32</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Gastroenterology</td>
      <td>INDICATION: ,  Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE:  ,</td>
    </tr>
    <tr>
      <th>16</th>
      <td>432</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis.  She ha...</td>
    </tr>
    <tr>
      <th>15</th>
      <td>604</td>
      <td>False</td>
      <td>Ophthalmology</td>
      <td>Consult - History and Phy.</td>
      <td>Her past medical history includes insulin requiring diabetes mellitus for the past 28 years.  She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease.  ...</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Nephrology</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>13</th>
      <td>558</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc...</td>
    </tr>
    <tr>
      <th>23</th>
      <td>616</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Respiratory failure.,POSTOPERATIVE DIAGNOSIS:  ,Respiratory failure.,OPERATIVE PROCEDURE: , Tracheotomy.,ANESTHESIA:  ,General inhalational.,DESCRIPTION OF PROCEDURE: , T...</td>
    </tr>
    <tr>
      <th>12</th>
      <td>913</td>
      <td>False</td>
      <td>ENT - Otolaryngology</td>
      <td>Hematology - Oncology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA:  ,Gene...</td>
    </tr>
    <tr>
      <th>10</th>
      <td>692</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS:  ,Rule out temporal arteritis.,PROCEDURE:,  Bilateral temporal artery biopsy.,ANESTHESIA:,  Local anesthesia 1% Xylo...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>104</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1.  Soft tissue mass, right knee.,2.  Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>754</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Ophthalmology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,POSTOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,PROCEDURE PERFORMED: , YAG laser capsulotomy, right eye.,INDICA...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Psychiatry / Psychology</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>228</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Consult - History and Phy.</td>
      <td>CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>250</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Neurosurgery</td>
      <td>REASON FOR VISIT:  ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:,  The pat...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>281</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT:  ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:,  This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>759</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Pain Management</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1.  Stellate ganglion RFTC (radiofrequency thermocoagulation)  left side.,2.  Interpret...</td>
    </tr>
    <tr>
      <th>2</th>
      <td>25</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Neurology</td>
      <td>DIAGNOSES:,1.  Juvenile myoclonic epilepsy.,2.  Recent generalized tonic-clonic seizure.,MEDICATIONS:,1.  Lamictal 250 mg b.i.d.,2.  Depo-Provera.,INTERIM HISTORY: , The patient returns for follow...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>114</td>
      <td>False</td>
      <td>Rheumatology</td>
      <td>Consult - History and Phy.</td>
      <td>HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria.  A was worked up for collagen vascular disease...</td>
    </tr>
    <tr>
      <th>24</th>
      <td>27</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>General Medicine</td>
      <td>REVIEW OF SYSTEMS:,CONSTITUTIONAL:  Patient denies fevers, chills, sweats and weight changes.,EYES:  Patient denies any visual symptoms.,EARS, NOSE, AND THROAT:  No difficulties with hearing.  No ...</td>
    </tr>
    <tr>
      <th>22</th>
      <td>517</td>
      <td>True</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>PROCEDURE PERFORMED:,1.  Left heart catheterization, left ventriculogram, aortogram, coronary angiogram.,2.  PCI of the LAD and left main coronary artery with Impella assist device.,INDICATIONS FO...</td>
    </tr>
    <tr>
      <th>11</th>
      <td>758</td>
      <td>True</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>SUBJECTIVE:,  He is a 24-year-old male who said that he had gotten into some poison ivy this weekend while he was fishing.  He has had several cases of this in the past and he says that is usually...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-0ee37ca0-1b4a-44f9-8d06-c8ae1a34dec3')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>

  </div>


  <div id="id_b16c813b-cc47-432f-a228-c65af46170ae">
    <style>
      .colab-df-generate {
        background-color: #E8F0FE;
        border: none;
        border-radius: 50%;
        cursor: pointer;
        display: none;
        fill: #1967D2;
        height: 32px;
        padding: 0 0 0 0;
        width: 32px;
      }

      .colab-df-generate:hover {
        background-color: #E2EBFA;
        box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
        fill: #174EA6;
      }

      [theme=dark] .colab-df-generate {
        background-color: #3B4455;
        fill: #D2E3FC;
      }

      [theme=dark] .colab-df-generate:hover {
        background-color: #434B5C;
        box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
        filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
        fill: #FFFFFF;
      }
    </style>
    <button class="colab-df-generate" onclick="generateWithVariable('hc_encoder_df')"
            title="Generate code using this dataframe."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px"viewBox="0 0 24 24"
       width="24px">
    <path d="M7,19H8.4L18.45,9,17,7.55,7,17.6ZM5,21V16.75L18.45,3.32a2,2,0,0,1,2.83,0l1.4,1.43a1.91,1.91,0,0,1,.58,1.4,1.91,1.91,0,0,1-.58,1.4L9.25,21ZM18.45,9,17,7.55Zm-12,3A5.31,5.31,0,0,0,4.9,8.1,5.31,5.31,0,0,0,1,6.5,5.31,5.31,0,0,0,4.9,4.9,5.31,5.31,0,0,0,6.5,1,5.31,5.31,0,0,0,8.1,4.9,5.31,5.31,0,0,0,12,6.5,5.46,5.46,0,0,0,6.5,12Z"/>
  </svg>
    </button>
    <script>
      (() => {
      const buttonEl =
        document.querySelector('#id_b16c813b-cc47-432f-a228-c65af46170ae button.colab-df-generate');
      buttonEl.style.display =
        google.colab.kernel.accessAllowed ? 'block' : 'none';

      buttonEl.onclick = () => {
        google.colab.notebook.generateWithVariable('hc_encoder_df');
      }
      })();
    </script>
  </div>

    </div>
  </div>

In [ ]:
hc_encoder_df[hc_encoder_df["correct"] == False].head(20)

,idx,correct,gold_label,pred_label,text
0,654,False,Gastroenterology,Surgery,"The patient's abdomen was prepped and draped in the usual sterile fashion. A subumbilical skin incision was made. The Veress needle was inserted, and the patient's abdominal cavity was insufflat..."
21,238,False,Discharge Summary,Consult - History and Phy.,"CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis..."
20,223,False,Orthopedic,Neurology,Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature. Motor strength was 5/5 on the MRC scale...
19,95,False,Pain Management,Orthopedic,"COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:, After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form. The patient was pl..."
18,30,False,Neurosurgery,Surgery,"PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:, MAC (monitored ane..."
17,32,False,Surgery,Gastroenterology,"INDICATION: , Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE: ,"
16,432,False,General Medicine,Consult - History and Phy.,"CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis. She ha..."
15,604,False,Ophthalmology,Consult - History and Phy.,Her past medical history includes insulin requiring diabetes mellitus for the past 28 years. She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease. ...
14,89,False,Consult - History and Phy.,Nephrology,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
13,558,False,Cardiovascular / Pulmonary,Surgery,"PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc..."


<div id="df-46cd379d-b5ea-4137-be81-a23f0479babb" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold_label</th>
      <th>pred_label</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>Surgery</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>21</th>
      <td>238</td>
      <td>False</td>
      <td>Discharge Summary</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis...</td>
    </tr>
    <tr>
      <th>20</th>
      <td>223</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Neurology</td>
      <td>Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature.  Motor strength was 5/5 on the MRC scale...</td>
    </tr>
    <tr>
      <th>19</th>
      <td>95</td>
      <td>False</td>
      <td>Pain Management</td>
      <td>Orthopedic</td>
      <td>COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:,  After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form.  The patient was pl...</td>
    </tr>
    <tr>
      <th>18</th>
      <td>30</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:,  MAC (monitored ane...</td>
    </tr>
    <tr>
      <th>17</th>
      <td>32</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Gastroenterology</td>
      <td>INDICATION: ,  Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE:  ,</td>
    </tr>
    <tr>
      <th>16</th>
      <td>432</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis.  She ha...</td>
    </tr>
    <tr>
      <th>15</th>
      <td>604</td>
      <td>False</td>
      <td>Ophthalmology</td>
      <td>Consult - History and Phy.</td>
      <td>Her past medical history includes insulin requiring diabetes mellitus for the past 28 years.  She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease.  ...</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Nephrology</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>13</th>
      <td>558</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc...</td>
    </tr>
    <tr>
      <th>23</th>
      <td>616</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Respiratory failure.,POSTOPERATIVE DIAGNOSIS:  ,Respiratory failure.,OPERATIVE PROCEDURE: , Tracheotomy.,ANESTHESIA:  ,General inhalational.,DESCRIPTION OF PROCEDURE: , T...</td>
    </tr>
    <tr>
      <th>12</th>
      <td>913</td>
      <td>False</td>
      <td>ENT - Otolaryngology</td>
      <td>Hematology - Oncology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA:  ,Gene...</td>
    </tr>
    <tr>
      <th>10</th>
      <td>692</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS:  ,Rule out temporal arteritis.,PROCEDURE:,  Bilateral temporal artery biopsy.,ANESTHESIA:,  Local anesthesia 1% Xylo...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>104</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1.  Soft tissue mass, right knee.,2.  Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>754</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Ophthalmology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,POSTOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,PROCEDURE PERFORMED: , YAG laser capsulotomy, right eye.,INDICA...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Psychiatry / Psychology</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>228</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Consult - History and Phy.</td>
      <td>CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>250</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Neurosurgery</td>
      <td>REASON FOR VISIT:  ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:,  The pat...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>281</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT:  ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:,  This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>759</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Pain Management</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1.  Stellate ganglion RFTC (radiofrequency thermocoagulation)  left side.,2.  Interpret...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-46cd379d-b5ea-4137-be81-a23f0479babb')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>
  </div>


    </div>
  </div>

In [ ]:
# immediately tells if the model collapses to a few classes
import numpy as np
from collections import Counter

preds = output.predictions.argmax(axis=1)
print(Counter(preds).most_common(10))

[(np.int64(1), 178), (np.int64(6), 129), (np.int64(16), 111), (np.int64(10), 102), (np.int64(14), 58), (np.int64(0), 57), (np.int64(2), 51), (np.int64(8), 46), (np.int64(11), 43), (np.int64(25), 42)]


In [ ]:
for i in range(10):
  label=output.predictions[i].argmax()
  if label==1:
      print("Predicted class: surgery")
      print ("Text: ",final_df["test"][i]["text"][0:500])
  if label==6:
      print("Predicted class: Consult - History and Phy.")
      print ("Text: ",final_df["test"][i]["text"][0:500])


Predicted class: Consult - History and Phy.
Text:  VITAL SIGNS: , Blood pressure *, pulse *, respirations *, temperature *.,GENERAL APPEARANCE:,  Alert and in no apparent distress, calm, cooperative, and communicative.,HEENT: , Eyes:  EOMI.  PERRLA.  Sclerae nonicteric.  No lesions of lids, lashes, brows, or conjunctivae noted.  Funduscopic examination unremarkable.  Ears:  Normal set, shape, TMs, canals and hearing.  Nose and Sinuses:  Negative.  Mouth, Tongue, Teeth, and Throat:  Negative except for dental work.,NECK: , Supple and pain free wi
Predicted class: Consult - History and Phy.
Text:  SUBJECTIVE: , The patient is a 60-year-old female, who complained of coughing during meals.  Her outpatient evaluation revealed a mild-to-moderate cognitive linguistic deficit, which was completed approximately 2 months ago.  The patient had a history of hypertension and TIA/stroke.  The patient denied history of heartburn and/or gastroesophageal reflux disorder.  A modified barium swallow stud

### Conclusion:

Without fine-tuning, the encoder model achieves a macro F1-score of **0.02** and shows a strong bias toward frequent classes, particularly surgical and consultation-related labels, as confirmed by manual inspection.

## Fine-tuning

We will fine-tune our model based on: https://huggingface.co/learn/llm-course/en/chapter3/3.

This fine-tuning comes down to allow weight updates from the BERT-medical encoder for our classification task. Therefore, in this setting, we allow the encoder and the head weights to be changed and tuned to our task.

### Model importation

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, set_seed

set_seed(42)

tokenizer = AutoTokenizer.from_pretrained("Charangan/MedBERT")
model = AutoModelForSequenceClassification.from_pretrained("Charangan/MedBERT",num_labels=40)

# This code is adapted from https://huggingface.co/transformers/v4.2.2/training.html?

# I am defreezing entire encoder
for param in model.base_model.parameters():
    param.requires_grad = True # I allow the weights to be updated

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Charangan/MedBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training arguments

In [ ]:
# This code is adapted from : https://huggingface.co/transformers/v4.2.2/training.html?utm_source=chatgpt.com
from transformers import TrainingArguments



training_args = TrainingArguments(
    output_dir='.',          # output directory
    num_train_epochs=3,              # total # of training epochs --> small, as we only train the head
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation 
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    fp16=True,                       # Enable mixed precision
    data_seed=42,
    seed=42,
    report_to="none",   # avoids wandb prompt
)

#### Training loops

In [ ]:
# This code is adapted from : https://huggingface.co/transformers/v4.2.2/training.html?utm_source=chatgpt.com
from transformers import Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=final_df_tokenized["train"],         # training dataset
    eval_dataset= final_df_tokenized["validation"],          # evaluation dataset
    data_collator=data_collator, # allows dynamical padding --> every batch will have the same lenghts, which is max_length of this batch
    compute_metrics=compute_metrics # added to return f1
)

trainer.train()

Step,Training Loss
500,2.450100
1000,1.728400


TrainOutput(global_step=1305, training_loss=1.9540909222716116, metrics={'train_runtime': 346.2249, 'train_samples_per_second': 30.11, 'train_steps_per_second': 3.769, 'total_flos': 2743868604211200.0, 'train_loss': 1.9540909222716116, 'epoch': 3.0})

### Evaluate accuracy

In [ ]:
trainer.evaluate(eval_dataset=final_df_tokenized["test"])

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.7231144905090332,
 'eval_accuracy': 0.2746478873239437,
 'eval_f1': 0.15263430216039917,
 'eval_precision': 0.1609912848654958,
 'eval_recall': 0.1548999931813748,
 'eval_runtime': 7.6068,
 'eval_samples_per_second': 130.672,
 'eval_steps_per_second': 8.282,
 'epoch': 3.0}

{'eval_loss': 1.7231144905090332, \
 'eval_accuracy': 0.2746478873239437,\
 'eval_f1': 0.15263430216039917,\
 'eval_precision': 0.1609912848654958,\
 'eval_recall': 0.1548999931813748,\
 'eval_runtime': 7.6068,\
 'eval_samples_per_second': 130.672,\
 'eval_steps_per_second': 8.282,\
 'epoch': 3.0}

#### Human check

In [ ]:
output=trainer.predict(final_df_tokenized["test"])

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
id2label = {v: k for k, v in label2id.items()}

In [ ]:
hc_encoder_fine_tuned_df = human_check_from_output(
    output=output,
    raw_test=final_df["test"],
    id2label=id2label,
    n=25,
    seed=42
)

hc_encoder_fine_tuned_df

Human check | n=25 | sample accuracy = 8.00%


,idx,correct,gold_label,pred_label,text
0,654,False,Gastroenterology,Surgery,"The patient's abdomen was prepped and draped in the usual sterile fashion. A subumbilical skin incision was made. The Veress needle was inserted, and the patient's abdominal cavity was insufflat..."
21,238,False,Discharge Summary,Consult - History and Phy.,"CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis..."
20,223,False,Orthopedic,Neurology,Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature. Motor strength was 5/5 on the MRC scale...
19,95,False,Pain Management,Orthopedic,"COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:, After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form. The patient was pl..."
18,30,False,Neurosurgery,Surgery,"PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:, MAC (monitored ane..."
17,32,False,Surgery,Gastroenterology,"INDICATION: , Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE: ,"
16,432,False,General Medicine,Consult - History and Phy.,"CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis. She ha..."
15,604,False,Ophthalmology,Consult - History and Phy.,Her past medical history includes insulin requiring diabetes mellitus for the past 28 years. She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease. ...
14,89,False,Consult - History and Phy.,Nephrology,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
13,558,False,Cardiovascular / Pulmonary,Surgery,"PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc..."


Human check | n=25 | sample accuracy = 8.00%

<div id="df-f1b8e16c-a61c-4915-b079-dd19c8c89a53" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold_label</th>
      <th>pred_label</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>Surgery</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>21</th>
      <td>238</td>
      <td>False</td>
      <td>Discharge Summary</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis...</td>
    </tr>
    <tr>
      <th>20</th>
      <td>223</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Neurology</td>
      <td>Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature.  Motor strength was 5/5 on the MRC scale...</td>
    </tr>
    <tr>
      <th>19</th>
      <td>95</td>
      <td>False</td>
      <td>Pain Management</td>
      <td>Orthopedic</td>
      <td>COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:,  After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form.  The patient was pl...</td>
    </tr>
    <tr>
      <th>18</th>
      <td>30</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:,  MAC (monitored ane...</td>
    </tr>
    <tr>
      <th>17</th>
      <td>32</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Gastroenterology</td>
      <td>INDICATION: ,  Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE:  ,</td>
    </tr>
    <tr>
      <th>16</th>
      <td>432</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis.  She ha...</td>
    </tr>
    <tr>
      <th>15</th>
      <td>604</td>
      <td>False</td>
      <td>Ophthalmology</td>
      <td>Consult - History and Phy.</td>
      <td>Her past medical history includes insulin requiring diabetes mellitus for the past 28 years.  She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease.  ...</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Nephrology</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>13</th>
      <td>558</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc...</td>
    </tr>
    <tr>
      <th>23</th>
      <td>616</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Respiratory failure.,POSTOPERATIVE DIAGNOSIS:  ,Respiratory failure.,OPERATIVE PROCEDURE: , Tracheotomy.,ANESTHESIA:  ,General inhalational.,DESCRIPTION OF PROCEDURE: , T...</td>
    </tr>
    <tr>
      <th>12</th>
      <td>913</td>
      <td>False</td>
      <td>ENT - Otolaryngology</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA:  ,Gene...</td>
    </tr>
    <tr>
      <th>10</th>
      <td>692</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS:  ,Rule out temporal arteritis.,PROCEDURE:,  Bilateral temporal artery biopsy.,ANESTHESIA:,  Local anesthesia 1% Xylo...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>104</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1.  Soft tissue mass, right knee.,2.  Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>754</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Ophthalmology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,POSTOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,PROCEDURE PERFORMED: , YAG laser capsulotomy, right eye.,INDICA...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Psychiatry / Psychology</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>228</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Consult - History and Phy.</td>
      <td>CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>250</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Neurosurgery</td>
      <td>REASON FOR VISIT:  ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:,  The pat...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>281</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT:  ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:,  This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>759</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Pain Management</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1.  Stellate ganglion RFTC (radiofrequency thermocoagulation)  left side.,2.  Interpret...</td>
    </tr>
    <tr>
      <th>2</th>
      <td>25</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Neurology</td>
      <td>DIAGNOSES:,1.  Juvenile myoclonic epilepsy.,2.  Recent generalized tonic-clonic seizure.,MEDICATIONS:,1.  Lamictal 250 mg b.i.d.,2.  Depo-Provera.,INTERIM HISTORY: , The patient returns for follow...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>114</td>
      <td>False</td>
      <td>Rheumatology</td>
      <td>Consult - History and Phy.</td>
      <td>HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria.  A was worked up for collagen vascular disease...</td>
    </tr>
    <tr>
      <th>24</th>
      <td>27</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>General Medicine</td>
      <td>REVIEW OF SYSTEMS:,CONSTITUTIONAL:  Patient denies fevers, chills, sweats and weight changes.,EYES:  Patient denies any visual symptoms.,EARS, NOSE, AND THROAT:  No difficulties with hearing.  No ...</td>
    </tr>
    <tr>
      <th>22</th>
      <td>517</td>
      <td>True</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>PROCEDURE PERFORMED:,1.  Left heart catheterization, left ventriculogram, aortogram, coronary angiogram.,2.  PCI of the LAD and left main coronary artery with Impella assist device.,INDICATIONS FO...</td>
    </tr>
    <tr>
      <th>11</th>
      <td>758</td>
      <td>True</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>SUBJECTIVE:,  He is a 24-year-old male who said that he had gotten into some poison ivy this weekend while he was fishing.  He has had several cases of this in the past and he says that is usually...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-f1b8e16c-a61c-4915-b079-dd19c8c89a53')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>

  </div>


  <div id="id_8dfcc3bb-ce63-417d-8eb8-43c99c95a97c">
    <style>
      .colab-df-generate {
        background-color: #E8F0FE;
        border: none;
        border-radius: 50%;
        cursor: pointer;
        display: none;
        fill: #1967D2;
        height: 32px;
        padding: 0 0 0 0;
        width: 32px;
      }

      .colab-df-generate:hover {
        background-color: #E2EBFA;
        box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
        fill: #174EA6;
      }

      [theme=dark] .colab-df-generate {
        background-color: #3B4455;
        fill: #D2E3FC;
      }

      [theme=dark] .colab-df-generate:hover {
        background-color: #434B5C;
        box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
        filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
        fill: #FFFFFF;
      }
    </style>
    <button class="colab-df-generate" onclick="generateWithVariable('hc_encoder_fine_tuned_df')"
            title="Generate code using this dataframe."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px"viewBox="0 0 24 24"
       width="24px">
    <path d="M7,19H8.4L18.45,9,17,7.55,7,17.6ZM5,21V16.75L18.45,3.32a2,2,0,0,1,2.83,0l1.4,1.43a1.91,1.91,0,0,1,.58,1.4,1.91,1.91,0,0,1-.58,1.4L9.25,21ZM18.45,9,17,7.55Zm-12,3A5.31,5.31,0,0,0,4.9,8.1,5.31,5.31,0,0,0,1,6.5,5.31,5.31,0,0,0,4.9,4.9,5.31,5.31,0,0,0,6.5,1,5.31,5.31,0,0,0,8.1,4.9,5.31,5.31,0,0,0,12,6.5,5.46,5.46,0,0,0,6.5,12Z"/>
  </svg>
    </button>
    <script>
      (() => {
      const buttonEl =
        document.querySelector('#id_8dfcc3bb-ce63-417d-8eb8-43c99c95a97c button.colab-df-generate');
      buttonEl.style.display =
        google.colab.kernel.accessAllowed ? 'block' : 'none';

      buttonEl.onclick = () => {
        google.colab.notebook.generateWithVariable('hc_encoder_fine_tuned_df');
      }
      })();
    </script>
  </div>

    </div>
  </div>

In [ ]:
hc_encoder_fine_tuned_df[hc_encoder_fine_tuned_df["correct"] == False].head(20)

,idx,correct,gold_label,pred_label,text
0,654,False,Gastroenterology,Surgery,"The patient's abdomen was prepped and draped in the usual sterile fashion. A subumbilical skin incision was made. The Veress needle was inserted, and the patient's abdominal cavity was insufflat..."
21,238,False,Discharge Summary,Consult - History and Phy.,"CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis..."
20,223,False,Orthopedic,Neurology,Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature. Motor strength was 5/5 on the MRC scale...
19,95,False,Pain Management,Orthopedic,"COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:, After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form. The patient was pl..."
18,30,False,Neurosurgery,Surgery,"PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:, MAC (monitored ane..."
17,32,False,Surgery,Gastroenterology,"INDICATION: , Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE: ,"
16,432,False,General Medicine,Consult - History and Phy.,"CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis. She ha..."
15,604,False,Ophthalmology,Consult - History and Phy.,Her past medical history includes insulin requiring diabetes mellitus for the past 28 years. She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease. ...
14,89,False,Consult - History and Phy.,Nephrology,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
13,558,False,Cardiovascular / Pulmonary,Surgery,"PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc..."


<div id="df-fe6dcab3-f2de-4469-b683-c43598447874" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold_label</th>
      <th>pred_label</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>Surgery</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>21</th>
      <td>238</td>
      <td>False</td>
      <td>Discharge Summary</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis...</td>
    </tr>
    <tr>
      <th>20</th>
      <td>223</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Neurology</td>
      <td>Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature.  Motor strength was 5/5 on the MRC scale...</td>
    </tr>
    <tr>
      <th>19</th>
      <td>95</td>
      <td>False</td>
      <td>Pain Management</td>
      <td>Orthopedic</td>
      <td>COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:,  After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form.  The patient was pl...</td>
    </tr>
    <tr>
      <th>18</th>
      <td>30</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:,  MAC (monitored ane...</td>
    </tr>
    <tr>
      <th>17</th>
      <td>32</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Gastroenterology</td>
      <td>INDICATION: ,  Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE:  ,</td>
    </tr>
    <tr>
      <th>16</th>
      <td>432</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis.  She ha...</td>
    </tr>
    <tr>
      <th>15</th>
      <td>604</td>
      <td>False</td>
      <td>Ophthalmology</td>
      <td>Consult - History and Phy.</td>
      <td>Her past medical history includes insulin requiring diabetes mellitus for the past 28 years.  She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease.  ...</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Nephrology</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>13</th>
      <td>558</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc...</td>
    </tr>
    <tr>
      <th>23</th>
      <td>616</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Respiratory failure.,POSTOPERATIVE DIAGNOSIS:  ,Respiratory failure.,OPERATIVE PROCEDURE: , Tracheotomy.,ANESTHESIA:  ,General inhalational.,DESCRIPTION OF PROCEDURE: , T...</td>
    </tr>
    <tr>
      <th>12</th>
      <td>913</td>
      <td>False</td>
      <td>ENT - Otolaryngology</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA:  ,Gene...</td>
    </tr>
    <tr>
      <th>10</th>
      <td>692</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS:  ,Rule out temporal arteritis.,PROCEDURE:,  Bilateral temporal artery biopsy.,ANESTHESIA:,  Local anesthesia 1% Xylo...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>104</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1.  Soft tissue mass, right knee.,2.  Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>754</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Ophthalmology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,POSTOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,PROCEDURE PERFORMED: , YAG laser capsulotomy, right eye.,INDICA...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Psychiatry / Psychology</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>228</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Consult - History and Phy.</td>
      <td>CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>250</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Neurosurgery</td>
      <td>REASON FOR VISIT:  ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:,  The pat...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>281</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Consult - History and Phy.</td>
      <td>CHIEF COMPLAINT:  ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:,  This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>759</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Pain Management</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1.  Stellate ganglion RFTC (radiofrequency thermocoagulation)  left side.,2.  Interpret...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-fe6dcab3-f2de-4469-b683-c43598447874')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>

  </div>


    </div>
  </div>

In [ ]:
for i in range(10):
  label=output.predictions[i].argmax()
  if label==2:
      print("Predicted class: Radiology")
      print ("Text: ",final_df["test"][i]["text"][0:500])
  if label==6:
      print("Predicted class: Consult - History and Phy.")
      print ("Text: ",final_df["test"][i]["text"][0:500])
  if label==8:
      print("Predicted class: General Medicine")
      print ("Text: ",final_df["test"][i]["text"][0:500])
  if label==10:
      print("Predicted class: Cardiovascular/ Pulmonary")
      print ("Text: ",final_df["test"][i]["text"][0:500])
  if label==12:
      print("Predicted class: Ophthalmology")
      print ("Text: ",final_df["test"][i]["text"][0:500])
  if label==16:
      print("Predicted class: Orthopedic")
      print ("Text: ",final_df["test"][i]["text"][0:500])
  if label==31:
      print("Predicted class: Discharge Summary")
      print ("Text: ",final_df["test"][i]["text"][0:500])

Predicted class: Orthopedic
Text:  PREOPERATIVE DIAGNOSES:,1.  Pathologic insufficiency.,2.  Fracture of the T8 vertebrae and T9 vertebrae.,POSTOPERATIVE DIAGNOSES:,1.  Pathologic insufficiency.,2.  Fracture of the T8 vertebra and T9 vertebra.,PROCEDURE PERFORMED:,1.  Fracture reduction with insertion of prosthetic device at T8 with kyphoplasty.,2.  Vertebroplasties at T7 and T9 with insertion of prosthetic device.,ANESTHESIA: , Local with sedation.,SPECIMEN: , Bone from the T8 vertebra.,COMPLICATIONS:,  None.,SURGICAL INDICATION
Predicted class: Discharge Summary
Text:  ADMISSION DIAGNOSES:,1.  Pneumonia, failed outpatient treatment.,2.  Hypoxia.,3.  Rheumatoid arthritis.,DISCHARGE DIAGNOSES:,1.  Atypical pneumonia, suspected viral.,2.  Hypoxia.,3.  Rheumatoid arthritis.,4.  Suspected mild stress-induced adrenal insufficiency.,HOSPITAL COURSE: , This very independent 79-year old had struggled with cough, fevers, weakness, and chills for the week prior to admission.  She was seen on mu

## Conclusion

We further fine-tuned the MedBERT model by allowing weight updates in both the encoder and the classification head. In contrast to the frozen-encoder setup, this enables the model to adapt its internal representations to the downstream task of fine-grained medical specialty classification.\
This full fine-tuning resulted in a substantial performance improvement, increasing the macro F1-score from 0.02 to 0.15, with corresponding gains in precision and recall. These results indicate that task-specific adaptation of the encoder is crucial for learning discriminative features in long and heterogeneous clinical notes, particularly when the label space is large and semantically nuanced.\
Manual inspection of the predictions shows that most remaining errors occur between semantically similar categories, especially closely related surgical specialties (e.g., Surgery, Neurosurgery, and Orthopedic), as well as clinically adjacent disciplines such as Neurology and Psychiatry / Psychology. This suggests that while the fine-tuned model captures high-level domain information, it still struggles with fine-grained distinctions between overlapping medical specialties. Additionally, truncation of clinical notes to 512 tokens may remove relevant contextual information, which can further contribute to misclassification.

# 3. Decoder track
Decoder-only large language models (LLMs) can be applied to classification tasks by reformulating classification as text generation. The model is prompted to generate the correct label given an input medical note.

We evaluate decoder models under these three settings:

Zero-shot prompting

Few-shot (5-shot) prompting

Parameter-Efficient Fine-Tuning (PEFT) using LoRA

All models are evaluated using macro-F1.

We use the decoder-only model **Qwen/Qwen2.5-0.5B-Instruct**, a compact instruction-tuned LLM. This model was selected as the smallest decoder model that could be reliably loaded and executed within our computational constraints, while still supporting instruction-following and text generation.

## 3.1 Model importation

In [122]:
from transformers import AutoTokenizer, AutoModelForCausalLM

set_seed(42)

model_id = "Qwen/Qwen2.5-0.5B-Instruct"

tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

def format_qwen_chat(user_content: str) -> str:
    messages = [
        {"role": "system", "content": "You are a medical coding assistant."},
        {"role": "user", "content": user_content},
    ]
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [79]:
print(label2id)

{'Gastroenterology': 0, 'Surgery': 1, 'Radiology': 2, 'SOAP / Chart / Progress Notes': 3, 'Letters': 4, 'Lab Medicine - Pathology': 5, 'Consult - History and Phy.': 6, 'Podiatry': 7, 'General Medicine': 8, 'Psychiatry / Psychology': 9, 'Cardiovascular / Pulmonary': 10, 'Urology': 11, 'Ophthalmology': 12, 'Physical Medicine - Rehab': 13, 'Neurology': 14, 'Autopsy': 15, 'Orthopedic': 16, 'Hematology - Oncology': 17, 'Allergy / Immunology': 18, 'Pediatrics - Neonatal': 19, 'Dentistry': 20, 'Neurosurgery': 21, 'Pain Management': 22, 'Nephrology': 23, 'Emergency Room Reports': 24, 'Obstetrics / Gynecology': 25, 'Speech - Language': 26, 'Diets and Nutritions': 27, 'Endocrinology': 28, 'IME-QME-Work Comp etc.': 29, 'Cosmetic / Plastic Surgery': 30, 'Discharge Summary': 31, 'ENT - Otolaryngology': 32, 'Chiropractic': 33, 'Office Notes': 34, 'Dermatology': 35, 'Sleep Medicine': 36, 'Rheumatology': 37, 'Hospice - Palliative Care': 38, 'Bariatrics': 39}


In [80]:
example_text = final_df["test"][0]["text"]
print(example_text)

PREOPERATIVE DIAGNOSES:,1.  Pathologic insufficiency.,2.  Fracture of the T8 vertebrae and T9 vertebrae.,POSTOPERATIVE DIAGNOSES:,1.  Pathologic insufficiency.,2.  Fracture of the T8 vertebra and T9 vertebra.,PROCEDURE PERFORMED:,1.  Fracture reduction with insertion of prosthetic device at T8 with kyphoplasty.,2.  Vertebroplasties at T7 and T9 with insertion of prosthetic device.,ANESTHESIA: , Local with sedation.,SPECIMEN: , Bone from the T8 vertebra.,COMPLICATIONS:,  None.,SURGICAL INDICATIONS:,  The patient is an 80-year-old female who had previous history of compression fractures.  She had recently undergone an additional compression fracture of the T8 vertebrae.  She was in extreme pain.  This pain interfered with activities of daily living and was unimproved with conservative treatment modalities.  She is understanding the risks, benefits, and potential complications as well as all treatment alternatives.  The patient provided informed consent.,OPERATIVE TECHNIQUE: , The patient

## 3.2 Zero shot prompting

We first evaluate the decoder model in a zero-shot prompting setting. In this setup, the model receives only a natural language instruction and the input medical note, without any examples.

We define a fixed prompt template that:
- instructs the model to act as a medical coding assistant,
- provides the complete list of possible labels,
- requires the model to output exactly one label from the list.

In [15]:
import re
import torch

set_seed(42)

LABELS = list(label2id.keys())  # or however you store your 40 labels
#LABELS_STR = "\n".join(f"- {lab}" for lab in LABELS)
LABELS_STR = "\n".join(LABELS)

def make_zeroshot_prompt(text: str) -> str:
    user_content = (
        "Choose EXACTLY ONE label from the list below that best matches the medical note.\n\n"
        f"Labels:\n{LABELS_STR}\n\n"
        f"Medical note:\n{text}\n\n"
        "Answer with exactly one label from the list and nothing else.\n"
        "Label:"
    )
    return format_qwen_chat(user_content)

In [ ]:
print(LABELS_STR)

Gastroenterology
Surgery
Radiology
SOAP / Chart / Progress Notes
Letters
Lab Medicine - Pathology
Consult - History and Phy.
Podiatry
General Medicine
Psychiatry / Psychology
Cardiovascular / Pulmonary
Urology
Ophthalmology
Physical Medicine - Rehab
Neurology
Autopsy
Orthopedic
Hematology - Oncology
Allergy / Immunology
Pediatrics - Neonatal
Dentistry
Neurosurgery
Pain Management
Nephrology
Emergency Room Reports
Obstetrics / Gynecology
Speech - Language
Diets and Nutritions
Endocrinology
IME-QME-Work Comp etc.
Cosmetic / Plastic Surgery
Discharge Summary
ENT - Otolaryngology
Chiropractic
Office Notes
Dermatology
Sleep Medicine
Rheumatology
Hospice - Palliative Care
Bariatrics


Next we define a function that performs zero-shot label prediction by generating a response from the decoder model and extracting the predicted label from the generated text.

In [ ]:
def predict_label_zeroshot(text: str, max_new_tokens: int = 8) -> str:
    prompt = make_zeroshot_prompt(text)

    inputs = tok(prompt, return_tensors="pt", truncation=True).to(model.device)
    input_len = inputs["input_ids"].shape[-1]
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,      # deterministic
            pad_token_id=tok.eos_token_id,
            eos_token_id=tok.eos_token_id,
        )

    decoded = tok.decode(out[0], skip_special_tokens=True)

    gen_ids = out[0][input_len:]
    raw = tok.decode(gen_ids, skip_special_tokens=True).strip()

    return raw

Because decoder models may generate additional text or formatting artifacts, we apply a normalization step that maps the generated output to one of the predefined labels. This includes removing punctuation and handling minor formatting variations. If the generated output does not match with any valid label, it is mapped to an `"UNKNOWN"` category.

In [ ]:
def normalize_to_label(pred_text: str, labels=LABELS) -> str:
    if pred_text is None:
        return "UNKNOWN"

    pred_text = pred_text.strip()

    # remove common bullet formatting
    pred_text = re.sub(r"^[\-\*\u2022]\s*", "", pred_text)

    # keep only first line (prevents matching label list / explanations)
    first_line = pred_text.splitlines()[0].strip()

    # clean punctuation
    cleaned = re.sub(r"[^A-Za-z0-9_\-/ ]+", "", first_line).strip()

    # exact match (case-insensitive)
    for lab in labels:
        if cleaned.lower() == lab.lower():
            return lab

    # startswith match (handles "Surgery because ...")
    for lab in labels:
        if cleaned.lower().startswith(lab.lower()):
            return lab


    return "UNKNOWN"

Example of raw and normalized decoder output

In [ ]:
raw = predict_label_zeroshot(example_text)
pred_label = normalize_to_label(raw)
print("RAW:", raw)
print("PRED:", pred_label)

RAW: Orthopedic
PRED: Orthopedic


In [ ]:
final_df["test"]

Dataset({
    features: ['text', 'labels'],
    num_rows: 994
})

Next we apply the zero-shot prompting procedure to the entire test set, generating one predicted label per medical note and collecting gold and predicted labels for evaluation.


In [ ]:
y_true, y_pred = [], []

for sample in final_df["test"]:
    text = sample["text"]
    gold = sample["labels"]

    raw = predict_label_zeroshot(text[:3000])
    pred = normalize_to_label(raw)

    y_true.append(gold)
    y_pred.append(pred)

In [ ]:
print(y_true)

[16, 10, 16, 34, 8, 17, 1, 10, 6, 0, 1, 1, 30, 0, 16, 1, 1, 2, 1, 16, 25, 6, 10, 11, 8, 34, 21, 34, 17, 9, 21, 10, 1, 21, 0, 19, 11, 25, 1, 11, 8, 6, 1, 17, 3, 1, 2, 1, 32, 14, 1, 2, 14, 32, 10, 34, 0, 6, 6, 10, 16, 1, 6, 24, 31, 10, 1, 7, 1, 6, 1, 6, 16, 13, 6, 22, 1, 3, 2, 16, 1, 21, 8, 14, 14, 1, 1, 1, 16, 6, 10, 0, 16, 6, 0, 22, 0, 1, 6, 10, 31, 1, 3, 10, 16, 10, 1, 2, 6, 10, 6, 3, 16, 22, 37, 1, 32, 16, 23, 2, 6, 1, 1, 17, 14, 1, 2, 3, 16, 2, 15, 28, 20, 31, 1, 16, 8, 3, 16, 25, 2, 14, 6, 32, 32, 32, 6, 1, 9, 16, 16, 7, 10, 22, 0, 1, 1, 3, 2, 6, 11, 1, 29, 1, 23, 10, 1, 22, 9, 0, 0, 10, 32, 22, 3, 21, 10, 23, 25, 11, 32, 1, 11, 21, 12, 8, 2, 31, 39, 1, 23, 2, 1, 1, 8, 24, 10, 10, 39, 16, 31, 6, 1, 1, 6, 6, 2, 1, 30, 24, 0, 16, 2, 12, 11, 17, 19, 1, 1, 12, 10, 16, 1, 16, 3, 1, 16, 6, 14, 1, 1, 9, 6, 1, 1, 12, 16, 1, 31, 27, 11, 25, 0, 14, 32, 1, 10, 1, 10, 1, 14, 10, 8, 1, 1, 2, 3, 10, 2, 0, 35, 1, 11, 6, 1, 31, 6, 20, 17, 1, 14, 2, 8, 1, 1, 1, 20, 10, 31, 14, 10, 8, 16, 31, 8, 21,

In [ ]:
id2label = {v: k for k, v in label2id.items()}

for i in range(len(y_true)):
    y_true[i] = id2label[y_true[i]]

In [ ]:
y_true

['Orthopedic',
 'Cardiovascular / Pulmonary',
 'Orthopedic',
 'Office Notes',
 'General Medicine',
 'Hematology - Oncology',
 'Surgery',
 'Cardiovascular / Pulmonary',
 'Consult - History and Phy.',
 'Gastroenterology',
 'Surgery',
 'Surgery',
 'Cosmetic / Plastic Surgery',
 'Gastroenterology',
 'Orthopedic',
 'Surgery',
 'Surgery',
 'Radiology',
 'Surgery',
 'Orthopedic',
 'Obstetrics / Gynecology',
 'Consult - History and Phy.',
 'Cardiovascular / Pulmonary',
 'Urology',
 'General Medicine',
 'Office Notes',
 'Neurosurgery',
 'Office Notes',
 'Hematology - Oncology',
 'Psychiatry / Psychology',
 'Neurosurgery',
 'Cardiovascular / Pulmonary',
 'Surgery',
 'Neurosurgery',
 'Gastroenterology',
 'Pediatrics - Neonatal',
 'Urology',
 'Obstetrics / Gynecology',
 'Surgery',
 'Urology',
 'General Medicine',
 'Consult - History and Phy.',
 'Surgery',
 'Hematology - Oncology',
 'SOAP / Chart / Progress Notes',
 'Surgery',
 'Radiology',
 'Surgery',
 'ENT - Otolaryngology',
 'Neurology',
 'Surge

In [ ]:
y_pred

['Orthopedic',
 'UNKNOWN',
 'Neurology',
 'UNKNOWN',
 'General Medicine',
 'Neurology',
 'Orthopedic',
 'Radiology',
 'Psychiatry / Psychology',
 'Speech - Language',
 'Orthopedic',
 'Orthopedic',
 'Orthopedic',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'Orthopedic',
 'UNKNOWN',
 'Orthopedic',
 'Orthopedic',
 'Orthopedic',
 'Chiropractic',
 'Emergency Room Reports',
 'Orthopedic',
 'UNKNOWN',
 'Neurology',
 'Orthopedic',
 'Orthopedic',
 'UNKNOWN',
 'Psychiatry / Psychology',
 'Orthopedic',
 'Cardiovascular / Pulmonary',
 'UNKNOWN',
 'UNKNOWN',
 'Orthopedic',
 'UNKNOWN',
 'Orthopedic',
 'Orthopedic',
 'Radiology',
 'Orthopedic',
 'Chiropractic',
 'General Medicine',
 'Orthopedic',
 'UNKNOWN',
 'Cardiovascular / Pulmonary',
 'Orthopedic',
 'Neurology',
 'Orthopedic',
 'Orthopedic',
 'UNKNOWN',
 'Orthopedic',
 'Radiology',
 'Neurology',
 'UNKNOWN',
 'UNKNOWN',
 'Orthopedic',
 'UNKNOWN',
 'General Medicine',
 'UNKNOWN',
 'UNKNOWN',
 'Orthopedic',
 'Orthopedic',
 'Gastroenterology',
 'Orthopedic

As a quick sanity check, we compute the exact-match accuracy over the test set.

In [ ]:
number_correct = 0

for i, j in zip(y_true, y_pred):
    if i == j:
        number_correct += 1

percent_correct = number_correct / len(y_true) * 100

print(percent_correct)

13.380281690140844


In [ ]:
final_df["train"]

Dataset({
    features: ['text', 'labels'],
    num_rows: 3475
})

We evaluate zero-shot predictions using the same metrics as in previous sections.

In [96]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred, y_true):
    labels = y_true
    preds = pred
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
compute_metrics(y_pred, y_true)

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.13380281690140844,
 'f1': 0.10664178205984462,
 'precision': 0.15084466728202786,
 'recall': 0.1376532894766879}

{'accuracy': 0.13380281690140844, \
 'f1': 0.10664178205984462,\
 'precision': 0.15084466728202786,\
 'recall': 0.1376532894766879}

In [60]:
import random
import pandas as pd

def human_check_qwen(
    final_test,
    y_true_labels,
    y_pred_labels,
    predict_raw_fn=None,
    normalize_fn=None,
    n=25,
    seed=42,
    text_chars=500,
    show_only_errors=False
):
    random.seed(seed)
    N = len(final_test)
    idxs = random.sample(range(N), k=min(n, N))

    rows = []
    for i in idxs:
        text = final_test[i]["text"]
        gold = y_true_labels[i]
        pred = y_pred_labels[i]
        correct = (gold == pred)

        raw = None
        pred_norm = None
        if predict_raw_fn is not None and normalize_fn is not None:
            raw = predict_raw_fn(text[:3000])
            pred_norm = normalize_fn(raw)

        rows.append({
            "idx": i,
            "correct": correct,
            "gold": gold,
            "pred": pred,
            "raw": raw,
            "pred_norm": pred_norm,
            "text": text[:text_chars].replace("\n", " ")
        })

    df = pd.DataFrame(rows).sort_values(["correct", "idx"], ascending=[True, True])

    if show_only_errors:
        df = df[df["correct"] == False]

    sample_acc = (df["correct"].mean() if len(df) else 0.0)
    print(f"Human check | n={len(df)} | sample accuracy = {sample_acc:.2%}")
    return df


In [ ]:
hc_qwen_raw = human_check_qwen(
    final_test=final_df["test"],
    y_true_labels=y_true,
    y_pred_labels=y_pred,
    predict_raw_fn=predict_label_zeroshot,
    normalize_fn=normalize_to_label,
    n=15,
    seed=42
)
hc_qwen_raw


Human check | n=15 | sample accuracy = 6.67%


,idx,correct,gold,pred,raw,pred_norm,text
2,25,False,Office Notes,Neurology,Neurology,Neurology,"DIAGNOSES:,1. Juvenile myoclonic epilepsy.,2. Recent generalized tonic-clonic seizure.,MEDICATIONS:,1. Lamictal 250 mg b.i.d.,2. Depo-Provera.,INTERIM HISTORY: , The patient returns for follow..."
14,89,False,Consult - History and Phy.,Emergency Room Reports,Emergency Room Reports,Emergency Room Reports,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
1,114,False,Rheumatology,Orthopedic,Orthopedic,Orthopedic,"HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria. A was worked up for collagen vascular disease..."
7,142,False,Consult - History and Phy.,UNKNOWN,Physician Order Documentation,UNKNOWN,"REASON FOR CONSULT:, Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land..."
6,228,False,Neurology,Orthopedic,Orthopedic,Orthopedic,"CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig..."
5,250,False,Neurology,UNKNOWN,Lumbar puncture,UNKNOWN,"REASON FOR VISIT: ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:, The pat..."
4,281,False,General Medicine,UNKNOWN,Physiology,UNKNOWN,"CHIEF COMPLAINT: ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:, This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we..."
13,558,False,Cardiovascular / Pulmonary,UNKNOWN,Postoperative Diagnosis,UNKNOWN,"PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc..."
0,654,False,Gastroenterology,UNKNOWN,Anatomy and Surgery,UNKNOWN,"The patient's abdomen was prepped and draped in the usual sterile fashion. A subumbilical skin incision was made. The Veress needle was inserted, and the patient's abdominal cavity was insufflat..."
10,692,False,Neurosurgery,Orthopedic,Orthopedic,Orthopedic,"PREOPERATIVE DIAGNOSIS:, Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS: ,Rule out temporal arteritis.,PROCEDURE:, Bilateral temporal artery biopsy.,ANESTHESIA:, Local anesthesia 1% Xylo..."


Human check | n=15 | sample accuracy = 6.67%

<div id="df-ac06097c-ad2e-4413-ac22-23e35dd08ad4" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold</th>
      <th>pred</th>
      <th>raw</th>
      <th>pred_norm</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2</th>
      <td>25</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Neurology</td>
      <td>Neurology</td>
      <td>Neurology</td>
      <td>DIAGNOSES:,1.  Juvenile myoclonic epilepsy.,2.  Recent generalized tonic-clonic seizure.,MEDICATIONS:,1.  Lamictal 250 mg b.i.d.,2.  Depo-Provera.,INTERIM HISTORY: , The patient returns for follow...</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Emergency Room Reports</td>
      <td>Emergency Room Reports</td>
      <td>Emergency Room Reports</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>114</td>
      <td>False</td>
      <td>Rheumatology</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria.  A was worked up for collagen vascular disease...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>UNKNOWN</td>
      <td>Physician Order Documentation</td>
      <td>UNKNOWN</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>228</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>250</td>
      <td>False</td>
      <td>Neurology</td>
      <td>UNKNOWN</td>
      <td>Lumbar puncture</td>
      <td>UNKNOWN</td>
      <td>REASON FOR VISIT:  ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:,  The pat...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>281</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>UNKNOWN</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>CHIEF COMPLAINT:  ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:,  This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we...</td>
    </tr>
    <tr>
      <th>13</th>
      <td>558</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>UNKNOWN</td>
      <td>Postoperative Diagnosis</td>
      <td>UNKNOWN</td>
      <td>PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc...</td>
    </tr>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>UNKNOWN</td>
      <td>Anatomy and Surgery</td>
      <td>UNKNOWN</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>10</th>
      <td>692</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS:  ,Rule out temporal arteritis.,PROCEDURE:,  Bilateral temporal artery biopsy.,ANESTHESIA:,  Local anesthesia 1% Xylo...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>754</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>PREOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,POSTOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,PROCEDURE PERFORMED: , YAG laser capsulotomy, right eye.,INDICA...</td>
    </tr>
    <tr>
      <th>11</th>
      <td>758</td>
      <td>False</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>UNKNOWN</td>
      <td>Poison ivy</td>
      <td>UNKNOWN</td>
      <td>SUBJECTIVE:,  He is a 24-year-old male who said that he had gotten into some poison ivy this weekend while he was fishing.  He has had several cases of this in the past and he says that is usually...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>759</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1.  Stellate ganglion RFTC (radiofrequency thermocoagulation)  left side.,2.  Interpret...</td>
    </tr>
    <tr>
      <th>12</th>
      <td>913</td>
      <td>False</td>
      <td>ENT - Otolaryngology</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA:  ,Gene...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>104</td>
      <td>True</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1.  Soft tissue mass, right knee.,2.  Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-ac06097c-ad2e-4413-ac22-23e35dd08ad4')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>


  </div>


  <div id="id_76161d94-2a58-4ca1-af95-9d022ab01231">
    <style>
      .colab-df-generate {
        background-color: #E8F0FE;
        border: none;
        border-radius: 50%;
        cursor: pointer;
        display: none;
        fill: #1967D2;
        height: 32px;
        padding: 0 0 0 0;
        width: 32px;
      }

      .colab-df-generate:hover {
        background-color: #E2EBFA;
        box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
        fill: #174EA6;
      }

      [theme=dark] .colab-df-generate {
        background-color: #3B4455;
        fill: #D2E3FC;
      }

      [theme=dark] .colab-df-generate:hover {
        background-color: #434B5C;
        box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
        filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
        fill: #FFFFFF;
      }
    </style>
    <button class="colab-df-generate" onclick="generateWithVariable('hc_qwen_raw')"
            title="Generate code using this dataframe."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px"viewBox="0 0 24 24"
       width="24px">
    <path d="M7,19H8.4L18.45,9,17,7.55,7,17.6ZM5,21V16.75L18.45,3.32a2,2,0,0,1,2.83,0l1.4,1.43a1.91,1.91,0,0,1,.58,1.4,1.91,1.91,0,0,1-.58,1.4L9.25,21ZM18.45,9,17,7.55Zm-12,3A5.31,5.31,0,0,0,4.9,8.1,5.31,5.31,0,0,0,1,6.5,5.31,5.31,0,0,0,4.9,4.9,5.31,5.31,0,0,0,6.5,1,5.31,5.31,0,0,0,8.1,4.9,5.31,5.31,0,0,0,12,6.5,5.46,5.46,0,0,0,6.5,12Z"/>
  </svg>
    </button>
    <script>
      (() => {
      const buttonEl =
        document.querySelector('#id_76161d94-2a58-4ca1-af95-9d022ab01231 button.colab-df-generate');
      buttonEl.style.display =
        google.colab.kernel.accessAllowed ? 'block' : 'none';

      buttonEl.onclick = () => {
        google.colab.notebook.generateWithVariable('hc_qwen_raw');
      }
      })();
    </script>
  </div>

    </div>
  </div>

In [ ]:
hc_qwen_raw[hc_qwen_raw["correct"] == False].head(10)

,idx,correct,gold,pred,raw,pred_norm,text
2,25,False,Office Notes,Neurology,Neurology,Neurology,"DIAGNOSES:,1. Juvenile myoclonic epilepsy.,2. Recent generalized tonic-clonic seizure.,MEDICATIONS:,1. Lamictal 250 mg b.i.d.,2. Depo-Provera.,INTERIM HISTORY: , The patient returns for follow..."
14,89,False,Consult - History and Phy.,Emergency Room Reports,Emergency Room Reports,Emergency Room Reports,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
1,114,False,Rheumatology,Orthopedic,Orthopedic,Orthopedic,"HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria. A was worked up for collagen vascular disease..."
7,142,False,Consult - History and Phy.,UNKNOWN,Physician Order Documentation,UNKNOWN,"REASON FOR CONSULT:, Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land..."
6,228,False,Neurology,Orthopedic,Orthopedic,Orthopedic,"CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig..."
5,250,False,Neurology,UNKNOWN,Lumbar puncture,UNKNOWN,"REASON FOR VISIT: ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:, The pat..."
4,281,False,General Medicine,UNKNOWN,Physiology,UNKNOWN,"CHIEF COMPLAINT: ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:, This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we..."
13,558,False,Cardiovascular / Pulmonary,UNKNOWN,Postoperative Diagnosis,UNKNOWN,"PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc..."
0,654,False,Gastroenterology,UNKNOWN,Anatomy and Surgery,UNKNOWN,"The patient's abdomen was prepped and draped in the usual sterile fashion. A subumbilical skin incision was made. The Veress needle was inserted, and the patient's abdominal cavity was insufflat..."
10,692,False,Neurosurgery,Orthopedic,Orthopedic,Orthopedic,"PREOPERATIVE DIAGNOSIS:, Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS: ,Rule out temporal arteritis.,PROCEDURE:, Bilateral temporal artery biopsy.,ANESTHESIA:, Local anesthesia 1% Xylo..."


<div id="df-7d9a818f-5c22-4899-b6cc-66b79a36ae80" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold</th>
      <th>pred</th>
      <th>raw</th>
      <th>pred_norm</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2</th>
      <td>25</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Neurology</td>
      <td>Neurology</td>
      <td>Neurology</td>
      <td>DIAGNOSES:,1.  Juvenile myoclonic epilepsy.,2.  Recent generalized tonic-clonic seizure.,MEDICATIONS:,1.  Lamictal 250 mg b.i.d.,2.  Depo-Provera.,INTERIM HISTORY: , The patient returns for follow...</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Emergency Room Reports</td>
      <td>Emergency Room Reports</td>
      <td>Emergency Room Reports</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>114</td>
      <td>False</td>
      <td>Rheumatology</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria.  A was worked up for collagen vascular disease...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>UNKNOWN</td>
      <td>Physician Order Documentation</td>
      <td>UNKNOWN</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>228</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>250</td>
      <td>False</td>
      <td>Neurology</td>
      <td>UNKNOWN</td>
      <td>Lumbar puncture</td>
      <td>UNKNOWN</td>
      <td>REASON FOR VISIT:  ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:,  The pat...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>281</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>UNKNOWN</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>CHIEF COMPLAINT:  ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:,  This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we...</td>
    </tr>
    <tr>
      <th>13</th>
      <td>558</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>UNKNOWN</td>
      <td>Postoperative Diagnosis</td>
      <td>UNKNOWN</td>
      <td>PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc...</td>
    </tr>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>UNKNOWN</td>
      <td>Anatomy and Surgery</td>
      <td>UNKNOWN</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>10</th>
      <td>692</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS:  ,Rule out temporal arteritis.,PROCEDURE:,  Bilateral temporal artery biopsy.,ANESTHESIA:,  Local anesthesia 1% Xylo...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-7d9a818f-5c22-4899-b6cc-66b79a36ae80')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>

  </div>


    </div>
  </div>

## Conclusion

The zero-shot decoder model achieves a macro F1-score of approximately 0.11, indicating moderate performance on this multi-class classification task. While it performs below the fine-tuned encoder-based model, it demonstrates the ability to assign plausible labels without task-specific training. Manual inspection shows a mix of correct and incorrect predictions, as well as occasional outputs outside the predefined label set.

## 3.3 Five-shots prompting

We next evaluate the decoder model using five-shot prompting. To construct the prompt, we select five labeled examples from the training set, which are included as demonstrations of the task.


In [133]:
examples_five_shot_raw = final_df["train"].shuffle(seed=42).select(range(5))

In [134]:
#label2id
id2label

{0: 'Gastroenterology',
 1: 'Surgery',
 2: 'Radiology',
 3: 'SOAP / Chart / Progress Notes',
 4: 'Letters',
 5: 'Lab Medicine - Pathology',
 6: 'Consult - History and Phy.',
 7: 'Podiatry',
 8: 'General Medicine',
 9: 'Psychiatry / Psychology',
 10: 'Cardiovascular / Pulmonary',
 11: 'Urology',
 12: 'Ophthalmology',
 13: 'Physical Medicine - Rehab',
 14: 'Neurology',
 15: 'Autopsy',
 16: 'Orthopedic',
 17: 'Hematology - Oncology',
 18: 'Allergy / Immunology',
 19: 'Pediatrics - Neonatal',
 20: 'Dentistry',
 21: 'Neurosurgery',
 22: 'Pain Management',
 23: 'Nephrology',
 24: 'Emergency Room Reports',
 25: 'Obstetrics / Gynecology',
 26: 'Speech - Language',
 27: 'Diets and Nutritions',
 28: 'Endocrinology',
 29: 'IME-QME-Work Comp etc.',
 30: 'Cosmetic / Plastic Surgery',
 31: 'Discharge Summary',
 32: 'ENT - Otolaryngology',
 33: 'Chiropractic',
 34: 'Office Notes',
 35: 'Dermatology',
 36: 'Sleep Medicine',
 37: 'Rheumatology',
 38: 'Hospice - Palliative Care',
 39: 'Bariatrics'}

In [135]:
examples_five_shot = []

for i in range(5):
    example = {"text": examples_five_shot_raw["text"][i], "label": id2label[examples_five_shot_raw["labels"][i]]}
    examples_five_shot.append(example)

In [136]:
examples_five_shot

[{'text': "REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been noticing increasing shortness of breath.  He also started having some abdominal pain; however, he continued about his regular activity until the other day when he passed out at home.  His wife called paramedics and he was brought to the emergency room.,The patient has had a workup at this time which shows bilateral pulmonary infarcts.  He has been started on heparin and we are asked to see him because of increasing BUN and creatinine.,The patient has no past history of any renal problems.  He feels that he has been in good health until this current episode.  His appetite has been good.  He denies swelling in his feet or ankles.  He denies chest pain.  He denies any problems with bowel habits.  He denies any unexplained weight loss.  He denies any recent change in bowel habits or recen

Now we evaluate the decoder model in a five-shot prompting setting. In this setup, the model receives a natural language instruction, the input medical note and five examples of text with labels.

We define a fixed prompt template that:
- instructs the model to act as a medical coding assistant,
- provides the complete list of possible labels,
- includes five labeled example mdeical notes,
- requires the model to output exactly one label from the list.

In [142]:
LABELS = list(label2id.keys())  # or however you store your 40 labels
#LABELS_STR = "\n".join(f"- {lab}" for lab in LABELS)
LABELS_STR = "\n".join(LABELS)

def make_fiveshot_prompt(text: str) -> str:
    shots = ""
    for ex in examples_five_shot:
        shots += (
            "Medical note:\n"
            f"{ex['text'][:600]}\n"
            f"Label: {ex['label']}\n\n"
        )
    user_content = (
        "Choose EXACTLY ONE label from the list below that best matches the medical note.\n\n"
        f"Labels:\n{LABELS_STR}\n\n"
        "Examples:\n"
        f"{shots}"
        f"Medical note:\n{text}\n\n"
        "Answer with exactly one label from the list and nothing else.\n"
        "Label:"
    )
    return format_qwen_chat(user_content)

Next we define a function that performs five-shot label prediction by generating a response from the decoder model and extracting the predicted label from the generated text.

In [143]:
def predict_label_fiveshot(text: str, max_new_tokens: int = 12) -> str:
    prompt = make_fiveshot_prompt(text)

    inputs = tok(prompt, return_tensors="pt", truncation=True).to(model.device)
    input_len = inputs["input_ids"].shape[-1]

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,      # deterministic
            pad_token_id=tok.eos_token_id,
            eos_token_id=tok.eos_token_id,
        )

    decoded = tok.decode(out[0], skip_special_tokens=True)

    # take only what comes after "Label:"
    #pred_text = decoded.split("Label:")[-1].strip()
    gen_ids = out[0][input_len:]
    raw = tok.decode(gen_ids, skip_special_tokens=True).strip()

    return raw

In [144]:
example_text

'PREOPERATIVE DIAGNOSES:,1.  Pathologic insufficiency.,2.  Fracture of the T8 vertebrae and T9 vertebrae.,POSTOPERATIVE DIAGNOSES:,1.  Pathologic insufficiency.,2.  Fracture of the T8 vertebra and T9 vertebra.,PROCEDURE PERFORMED:,1.  Fracture reduction with insertion of prosthetic device at T8 with kyphoplasty.,2.  Vertebroplasties at T7 and T9 with insertion of prosthetic device.,ANESTHESIA: , Local with sedation.,SPECIMEN: , Bone from the T8 vertebra.,COMPLICATIONS:,  None.,SURGICAL INDICATIONS:,  The patient is an 80-year-old female who had previous history of compression fractures.  She had recently undergone an additional compression fracture of the T8 vertebrae.  She was in extreme pain.  This pain interfered with activities of daily living and was unimproved with conservative treatment modalities.  She is understanding the risks, benefits, and potential complications as well as all treatment alternatives.  The patient provided informed consent.,OPERATIVE TECHNIQUE: , The patien

Example of raw and normalized decoder output

In [145]:
raw = predict_label_fiveshot(example_text)
pred_label = normalize_to_label(raw)
print("RAW:", raw)
print("PRED:", pred_label)

RAW: Urology
PRED: Urology


RAW: Urology
PRED: Urology

Next we apply the five-shot prompting procedure to the entire test set, generating one predicted label per medical note and collecting gold and predicted labels for evaluation.

In [156]:
from tqdm import tqdm

y_true_fiveshot, y_pred_fiveshot = [], []

for sample in tqdm(final_df["test"], desc="Five-shot inference"):
    text = sample["text"]
    gold = sample["labels"]

    raw = predict_label_fiveshot(text[:3000])
    pred = raw#normalize_to_label(raw)

    y_true_fiveshot.append(gold)
    y_pred_fiveshot.append(pred)


Five-shot inference: 100%|██████████| 994/994 [12:06<00:00,  1.37it/s]


We evaluate five-shot predictions using the same evaluation metrics as before.

In [157]:
id2label = {v: k for k, v in label2id.items()}

for i in range(len(y_true_fiveshot)):
    y_true_fiveshot[i] = id2label[y_true_fiveshot[i]]

In [158]:
compute_metrics(y_pred_fiveshot, y_true_fiveshot)

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.03219315895372234,
 'f1': 0.005365269045274784,
 'precision': 0.01295113314296787,
 'recall': 0.023534997531069524}

{'accuracy': 0.03219315895372234, \
 'f1': 0.005365269045274784,\
 'precision': 0.01295113314296787,\
 'recall': 0.023534997531069524}

In [159]:
hc_qwen_five_raw = human_check_qwen(
    final_test=final_df["test"],
    y_true_labels=y_true_fiveshot,
    y_pred_labels=y_pred_fiveshot,
    predict_raw_fn=predict_label_fiveshot,
    normalize_fn=normalize_to_label,
    n=25,
    seed=42
)
hc_qwen_five_raw


Human check | n=25 | sample accuracy = 0.00%


,idx,correct,gold,pred,raw,pred_norm,text
2,25,False,Office Notes,Physician Order Documentation,Physician Order Documentation,UNKNOWN,"DIAGNOSES:,1. Juvenile myoclonic epilepsy.,2. Recent generalized tonic-clonic seizure.,MEDICATIONS:,1. Lamictal 250 mg b.i.d.,2. Depo-Provera.,INTERIM HISTORY: , The patient returns for follow..."
24,27,False,Office Notes,Physiology,Physiology,UNKNOWN,"REVIEW OF SYSTEMS:,CONSTITUTIONAL: Patient denies fevers, chills, sweats and weight changes.,EYES: Patient denies any visual symptoms.,EARS, NOSE, AND THROAT: No difficulties with hearing. No ..."
18,30,False,Neurosurgery,Physiology,Physiology,UNKNOWN,"PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:, MAC (monitored ane..."
17,32,False,Surgery,Urology,Urology,Urology,"INDICATION: , Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE: ,"
14,89,False,Consult - History and Phy.,Physiology,Physiology,UNKNOWN,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
19,95,False,Pain Management,Physiology,Physiology,UNKNOWN,"COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:, After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form. The patient was pl..."
9,104,False,Orthopedic,Urology,Urology,Urology,"PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1. Soft tissue mass, right knee.,2. Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio..."
1,114,False,Rheumatology,Urology,Urology,Urology,"HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria. A was worked up for collagen vascular disease..."
7,142,False,Consult - History and Phy.,Psychiatry / Psychology,Psychiatry / Psychology,Psychiatry / Psychology,"REASON FOR CONSULT:, Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land..."
20,223,False,Orthopedic,Urology,Urology,Urology,Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature. Motor strength was 5/5 on the MRC scale...


Human check | n=25 | sample accuracy = 0.00%

<div id="df-2a6ffe09-eeed-46c9-bda7-5178f24bf1fd" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold</th>
      <th>pred</th>
      <th>raw</th>
      <th>pred_norm</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2</th>
      <td>25</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Physician Order Documentation</td>
      <td>Physician Order Documentation</td>
      <td>UNKNOWN</td>
      <td>DIAGNOSES:,1.  Juvenile myoclonic epilepsy.,2.  Recent generalized tonic-clonic seizure.,MEDICATIONS:,1.  Lamictal 250 mg b.i.d.,2.  Depo-Provera.,INTERIM HISTORY: , The patient returns for follow...</td>
    </tr>
    <tr>
      <th>24</th>
      <td>27</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>REVIEW OF SYSTEMS:,CONSTITUTIONAL:  Patient denies fevers, chills, sweats and weight changes.,EYES:  Patient denies any visual symptoms.,EARS, NOSE, AND THROAT:  No difficulties with hearing.  No ...</td>
    </tr>
    <tr>
      <th>18</th>
      <td>30</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:,  MAC (monitored ane...</td>
    </tr>
    <tr>
      <th>17</th>
      <td>32</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>INDICATION: ,  Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE:  ,</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>19</th>
      <td>95</td>
      <td>False</td>
      <td>Pain Management</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:,  After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form.  The patient was pl...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>104</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1.  Soft tissue mass, right knee.,2.  Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>114</td>
      <td>False</td>
      <td>Rheumatology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria.  A was worked up for collagen vascular disease...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Psychiatry / Psychology</td>
      <td>Psychiatry / Psychology</td>
      <td>Psychiatry / Psychology</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>20</th>
      <td>223</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature.  Motor strength was 5/5 on the MRC scale...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>228</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig...</td>
    </tr>
    <tr>
      <th>21</th>
      <td>238</td>
      <td>False</td>
      <td>Discharge Summary</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>CHIEF COMPLAINT: , Chest pain and fever.,HISTORY OF PRESENT ILLNESS: , This 48-year-old white married female presents in the emergency room after two days of increasing fever with recent diagnosis...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>250</td>
      <td>False</td>
      <td>Neurology</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>REASON FOR VISIT:  ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:,  The pat...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>281</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>CHIEF COMPLAINT:  ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:,  This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we...</td>
    </tr>
    <tr>
      <th>16</th>
      <td>432</td>
      <td>False</td>
      <td>General Medicine</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>CHIEF COMPLAINT: , Swelling of lips causing difficulty swallowing.,HISTORY OF PRESENT ILLNESS:, This patient is a 57-year old white Cuban woman with a long history of rheumatoid arthritis.  She ha...</td>
    </tr>
    <tr>
      <th>22</th>
      <td>517</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>PROCEDURE PERFORMED:,1.  Left heart catheterization, left ventriculogram, aortogram, coronary angiogram.,2.  PCI of the LAD and left main coronary artery with Impella assist device.,INDICATIONS FO...</td>
    </tr>
    <tr>
      <th>13</th>
      <td>558</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc...</td>
    </tr>
    <tr>
      <th>15</th>
      <td>604</td>
      <td>False</td>
      <td>Ophthalmology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Her past medical history includes insulin requiring diabetes mellitus for the past 28 years.  She also has a history of gastritis and currently is being evaluated for inflammatory bowel disease.  ...</td>
    </tr>
    <tr>
      <th>23</th>
      <td>616</td>
      <td>False</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>PREOPERATIVE DIAGNOSIS: , Respiratory failure.,POSTOPERATIVE DIAGNOSIS:  ,Respiratory failure.,OPERATIVE PROCEDURE: , Tracheotomy.,ANESTHESIA:  ,General inhalational.,DESCRIPTION OF PROCEDURE: , T...</td>
    </tr>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>10</th>
      <td>692</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS:  ,Rule out temporal arteritis.,PROCEDURE:,  Bilateral temporal artery biopsy.,ANESTHESIA:,  Local anesthesia 1% Xylo...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>754</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,POSTOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,PROCEDURE PERFORMED: , YAG laser capsulotomy, right eye.,INDICA...</td>
    </tr>
    <tr>
      <th>11</th>
      <td>758</td>
      <td>False</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>Psychiatry / Psychology</td>
      <td>Psychiatry / Psychology</td>
      <td>Psychiatry / Psychology</td>
      <td>SUBJECTIVE:,  He is a 24-year-old male who said that he had gotten into some poison ivy this weekend while he was fishing.  He has had several cases of this in the past and he says that is usually...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>759</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1.  Stellate ganglion RFTC (radiofrequency thermocoagulation)  left side.,2.  Interpret...</td>
    </tr>
    <tr>
      <th>12</th>
      <td>913</td>
      <td>False</td>
      <td>ENT - Otolaryngology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA:  ,Gene...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-2a6ffe09-eeed-46c9-bda7-5178f24bf1fd')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>

  </div>


  <div id="id_ef1b8d30-1f21-4300-9895-43847e9edf5e">
    <style>
      .colab-df-generate {
        background-color: #E8F0FE;
        border: none;
        border-radius: 50%;
        cursor: pointer;
        display: none;
        fill: #1967D2;
        height: 32px;
        padding: 0 0 0 0;
        width: 32px;
      }

      .colab-df-generate:hover {
        background-color: #E2EBFA;
        box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
        fill: #174EA6;
      }

      [theme=dark] .colab-df-generate {
        background-color: #3B4455;
        fill: #D2E3FC;
      }

      [theme=dark] .colab-df-generate:hover {
        background-color: #434B5C;
        box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
        filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
        fill: #FFFFFF;
      }
    </style>
    <button class="colab-df-generate" onclick="generateWithVariable('hc_qwen_five_raw')"
            title="Generate code using this dataframe."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px"viewBox="0 0 24 24"
       width="24px">
    <path d="M7,19H8.4L18.45,9,17,7.55,7,17.6ZM5,21V16.75L18.45,3.32a2,2,0,0,1,2.83,0l1.4,1.43a1.91,1.91,0,0,1,.58,1.4,1.91,1.91,0,0,1-.58,1.4L9.25,21ZM18.45,9,17,7.55Zm-12,3A5.31,5.31,0,0,0,4.9,8.1,5.31,5.31,0,0,0,1,6.5,5.31,5.31,0,0,0,4.9,4.9,5.31,5.31,0,0,0,6.5,1,5.31,5.31,0,0,0,8.1,4.9,5.31,5.31,0,0,0,12,6.5,5.46,5.46,0,0,0,6.5,12Z"/>
  </svg>
    </button>
    <script>
      (() => {
      const buttonEl =
        document.querySelector('#id_ef1b8d30-1f21-4300-9895-43847e9edf5e button.colab-df-generate');
      buttonEl.style.display =
        google.colab.kernel.accessAllowed ? 'block' : 'none';

      buttonEl.onclick = () => {
        google.colab.notebook.generateWithVariable('hc_qwen_five_raw');
      }
      })();
    </script>
  </div>

    </div>
  </div>

In [160]:

hc_qwen_five_raw[hc_qwen_five_raw["correct"] == False].head(10)

,idx,correct,gold,pred,raw,pred_norm,text
2,25,False,Office Notes,Physician Order Documentation,Physician Order Documentation,UNKNOWN,"DIAGNOSES:,1. Juvenile myoclonic epilepsy.,2. Recent generalized tonic-clonic seizure.,MEDICATIONS:,1. Lamictal 250 mg b.i.d.,2. Depo-Provera.,INTERIM HISTORY: , The patient returns for follow..."
24,27,False,Office Notes,Physiology,Physiology,UNKNOWN,"REVIEW OF SYSTEMS:,CONSTITUTIONAL: Patient denies fevers, chills, sweats and weight changes.,EYES: Patient denies any visual symptoms.,EARS, NOSE, AND THROAT: No difficulties with hearing. No ..."
18,30,False,Neurosurgery,Physiology,Physiology,UNKNOWN,"PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:, MAC (monitored ane..."
17,32,False,Surgery,Urology,Urology,Urology,"INDICATION: , Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE: ,"
14,89,False,Consult - History and Phy.,Physiology,Physiology,UNKNOWN,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
19,95,False,Pain Management,Physiology,Physiology,UNKNOWN,"COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:, After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form. The patient was pl..."
9,104,False,Orthopedic,Urology,Urology,Urology,"PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1. Soft tissue mass, right knee.,2. Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio..."
1,114,False,Rheumatology,Urology,Urology,Urology,"HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria. A was worked up for collagen vascular disease..."
7,142,False,Consult - History and Phy.,Psychiatry / Psychology,Psychiatry / Psychology,Psychiatry / Psychology,"REASON FOR CONSULT:, Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land..."
20,223,False,Orthopedic,Urology,Urology,Urology,Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature. Motor strength was 5/5 on the MRC scale...


<div id="df-b0df36da-c35a-4ddd-84fb-6cb10dbcbfdb" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold</th>
      <th>pred</th>
      <th>raw</th>
      <th>pred_norm</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2</th>
      <td>25</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Physician Order Documentation</td>
      <td>Physician Order Documentation</td>
      <td>UNKNOWN</td>
      <td>DIAGNOSES:,1.  Juvenile myoclonic epilepsy.,2.  Recent generalized tonic-clonic seizure.,MEDICATIONS:,1.  Lamictal 250 mg b.i.d.,2.  Depo-Provera.,INTERIM HISTORY: , The patient returns for follow...</td>
    </tr>
    <tr>
      <th>24</th>
      <td>27</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>REVIEW OF SYSTEMS:,CONSTITUTIONAL:  Patient denies fevers, chills, sweats and weight changes.,EYES:  Patient denies any visual symptoms.,EARS, NOSE, AND THROAT:  No difficulties with hearing.  No ...</td>
    </tr>
    <tr>
      <th>18</th>
      <td>30</td>
      <td>False</td>
      <td>Neurosurgery</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>PREOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,POSTOPERATIVE DIAGNOSIS: , Tremor, dystonic form.,COMPLICATIONS: , None.,ESTIMATED BLOOD LOSS: , Less than 100 mL.,ANESTHESIA:,  MAC (monitored ane...</td>
    </tr>
    <tr>
      <th>17</th>
      <td>32</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>INDICATION: ,  Rectal bleeding, constipation, abnormal CT scan, rule out inflammatory bowel disease.,PREMEDICATION: ,See procedure nurse NCS form.,PROCEDURE:  ,</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>19</th>
      <td>95</td>
      <td>False</td>
      <td>Pain Management</td>
      <td>Physiology</td>
      <td>Physiology</td>
      <td>UNKNOWN</td>
      <td>COSTOCHONDRAL CARTILAGE INJECTION,PROCEDURE PREPARATION:,  After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form.  The patient was pl...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>104</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1.  Soft tissue mass, right knee.,2.  Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>114</td>
      <td>False</td>
      <td>Rheumatology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria.  A was worked up for collagen vascular disease...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Psychiatry / Psychology</td>
      <td>Psychiatry / Psychology</td>
      <td>Psychiatry / Psychology</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>20</th>
      <td>223</td>
      <td>False</td>
      <td>Orthopedic</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Urology</td>
      <td>Her evaluation today reveals restriction in the range of motion of the cervical and lumbar region with tenderness and spasms of the paraspinal musculature.  Motor strength was 5/5 on the MRC scale...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-b0df36da-c35a-4ddd-84fb-6cb10dbcbfdb')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>

  </div>


    </div>
  </div>

### Conclusion
In the five-shot prompting setting, the decoder model performs poorly (macro-F1 ≈ 0.005, accuracy ≈ 0.032). Manual inspection shows that the model repeatedly outputs a small set of responses (e.g., “Physiology” or “Urology”) \
This may be due to the limited capacity of the relatively small decoder model, combined with very long and heterogeneous in-context examples. The resulting prompt length likely overwhelms the model, making it difficult to identify and abstract the relevant signal from the few-shot demonstrations and effectively map them to the large label space.


## 3.4 Fine-tunning: PEFT SFT using Lora

Finally, we evaluate a supervised decoder-based approach using LoRA. LoRA freezes the base decoder model and trains a small number of additional parameters injected into the attention layers.

We define a fixed prompt template that:
- instructs the model to act as a medical coding assistant,
- provides the complete list of possible labels,
- requires the model to output exactly one label from the list.

In [16]:
from peft import LoraConfig, get_peft_model, TaskType

set_seed(42)
LABELS = list(label2id.keys())
LABELS_STR = "\n".join(LABELS)  # ohne "- "
def make_prompt(text: str) -> str:
    text = text[:1500]
    return (
        "Choose EXACTLY ONE label from the list below that best matches the medical note.\n\n"
        f"Labels:\n{LABELS_STR}\n\n"
        f"Medical note:\n{text}\n\n"
        "Answer with exactly one label from the list and nothing else.\n"
        "Label:"
    )

In [17]:
final_df["train"]

Dataset({
    features: ['text', 'labels'],
    num_rows: 3475
})

We initialize the base decoder model and tokenizer for supervised fine-tuning. Padding tokens are configured to ensure correct batching during sequence-to-sequence training.

In [36]:
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForCausalLM

token = AutoTokenizer.from_pretrained(model_id, use_fast=True)

if token.pad_token is None:
    token.pad_token = token.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, device_map="auto")

We configure LoRA to inject trainable adapters into layers of the decoder model.

In [37]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

We freeze the pretrained base model and inject LoRA adapter parameters.

In [38]:
model = get_peft_model(base_model, lora_config) # freeze pretrained base model and inject lora parameters
model.print_trainable_parameters() # see how many trainable parameters we have

trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184


trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184

To train the decoder model in a supervised manner, we tokenize the prompt–response pairs and construct labels such that the loss is computed only on the generated label tokens, while the prompt tokens are ignored.

In [39]:
def tokenize_prompt_response(example):
    prompt_ids = token(example["prompt"], add_special_tokens=False).input_ids # tokenize prompt
    resp_ids = token(" " + example["response"], add_special_tokens=False).input_ids # tokenize response

    input_ids = prompt_ids + resp_ids
    attention_mask = [1] * len(input_ids) # attend to whole input ([1])

    labels = [-100] * len(prompt_ids) + resp_ids # compute loss for labels, only tokens after medical text and instruction (label) is used for learning, no loss for predicting inside medical text

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

We convert each labeled training example into a prompt–response pair.

In [40]:
def prompt_plus_response(example):
    return {
        "prompt": make_prompt(example["text"]),
        "response": id2label[int(example["labels"])],  # id to label
    }

We apply the prompt–response transformation to the training, validation, and test splits to prepare data for supervised decoder fine-tuning and evaluation.

In [41]:


train_pairs = final_df["train"].map(prompt_plus_response)
val_pairs = final_df["validation"].map(prompt_plus_response)
test_pairs  = final_df["test"].map(prompt_plus_response)

Map:   0%|          | 0/3475 [00:00<?, ? examples/s]

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Map:   0%|          | 0/994 [00:00<?, ? examples/s]

In [42]:
test_pairs

Dataset({
    features: ['text', 'labels', 'prompt', 'response'],
    num_rows: 994
})

We tokenize the prompt–response pairs and construct the final input representations required for supervised decoder fine-tuning.

In [43]:
train_pairs_tokenized = []
val_pairs_tokenized = []
test_pairs_tokenized = []

for example in train_pairs:
    tokenized_example = tokenize_prompt_response(example)
    train_pairs_tokenized.append(tokenized_example)

for example in val_pairs:
    tokenized_example = tokenize_prompt_response(example)
    val_pairs_tokenized.append(tokenized_example)

for example in test_pairs:
    tokenized_example = tokenize_prompt_response(example)
    test_pairs_tokenized.append(tokenized_example)

In [44]:
ex = train_pairs_tokenized[0]
print("len(input_ids):", len(ex["input_ids"])) # length of input in tokens
print("len(attention_mask):", len(ex["attention_mask"])) # which tokens are "real" (no padding)
print("len(labels):", len(ex["labels"]))
# during training, the model predicts the next token everywhere, but loss and gradient update occur only at positions whose labels are not -100 (label tokens), so the model learns to generate the correct label given the prompt

len(input_ids): 597
len(attention_mask): 597
len(labels): 597


In [45]:
ex
print(ex["labels"][-10:]) # token ids of label are non -100, during training the label is supervised to predict these label tokens via next token prediction, with the prompt tokens ignored in the loss

[-100, -100, -100, -100, 63232, 608, 21266, 608, 16033, 18068]


In [46]:
# decode back into words to check
resp_token_ids = [y for y in ex["labels"] if y != -100]
print("num response tokens:", len(resp_token_ids))
print("decoded response:", tok.decode(resp_token_ids))

num response tokens: 6
decoded response:  SOAP / Chart / Progress Notes


In [47]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=token,
    padding=True,
    label_pad_token_id=-100,
    return_tensors="pt",
)

We specify training hyperparameters for LoRA fine-tuning of the decoder model, including learning rate, batch size, and number of training epochs.

In [49]:
from transformers import TrainingArguments

model.config.use_cache = False

training_args = TrainingArguments(
    output_dir="./qwen_lora_sft",
    num_train_epochs=3,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    warmup_steps=100, # stabilize early training
    weight_decay=0.01,
    seed=42,
    data_seed=42,
    remove_unused_columns=False,
    report_to="none",
    fp16=True,
)

In [50]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_pairs_tokenized,
    eval_dataset=val_pairs_tokenized,
    data_collator=data_collator,
    tokenizer=token,
)

/tmp/ipython-input-3586421603.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [51]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
500,0.687500
1000,0.484900


TrainOutput(global_step=1305, training_loss=0.5512299753240242, metrics={'train_runtime': 2105.1132, 'train_samples_per_second': 4.952, 'train_steps_per_second': 0.62, 'total_flos': 1.301608173393792e+16, 'train_loss': 0.5512299753240242, 'epoch': 3.0})

We evaluate the LoRA-fine-tuned decoder model on the test set by generating predicted labels for each medical note and collecting gold and predicted labels for evaluation.


In [69]:
id2label = {v: k for k, v in label2id.items()}

y_true_lbl, y_pred_lbl, y_raw = [], [], []
model.eval()

for sample in final_df["test"]:
    prompt = make_prompt(sample["text"])

    inputs = token(prompt, return_tensors="pt", truncation=True).to(model.device)
    in_len = inputs["input_ids"].shape[-1]

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=6,
            do_sample=False,
            pad_token_id=token.pad_token_id,
            eos_token_id=token.eos_token_id,
        )

    gen_ids = output_ids[0, in_len:]
    raw = token.decode(gen_ids, skip_special_tokens=True).strip()
    pred = normalize_to_label(raw)

    y_raw.append(raw)
    y_pred_lbl.append(pred)
    y_true_lbl.append(id2label[int(sample["labels"])])


We evaluate LoRA predictions using the same evaluation metrics as before.

In [99]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred, y_true):
    labels = y_true
    preds = pred
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [71]:
compute_metrics(y_pred_lbl, y_true_lbl)

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.3118712273641851,
 'f1': 0.21048160645029326,
 'precision': 0.22448870647354477,
 'recall': 0.23676427422548393}

{'accuracy': 0.3118712273641851,\
 'f1': 0.21048160645029326,\
 'precision': 0.22448870647354477,\
 'recall': 0.23676427422548393}

In [72]:
import random, pandas as pd

def human_check_precomputed(final_test, y_true, y_pred, raw=None, pred_norm=None,
                            n=25, seed=42, text_chars=500):
    random.seed(seed)
    N = len(final_test)
    idxs = random.sample(range(N), k=min(n, N))

    rows = []
    for i in idxs:
        rows.append({
            "idx": i,
            "correct": y_true[i] == y_pred[i],
            "gold": y_true[i],
            "pred": y_pred[i],
            "raw": (raw[i] if raw is not None else None),
            "pred_norm": (pred_norm[i] if pred_norm is not None else None),
            "text": final_test[i]["text"][:text_chars].replace("\n", " ")
        })

    df = pd.DataFrame(rows).sort_values(["correct", "idx"], ascending=[True, True])

    print(f"Human check | n={len(df)} | sample accuracy = {df['correct'].mean():.2%}")
    return df


In [76]:
hc_lora = human_check_precomputed(
    final_test=final_df["test"],
    y_true=y_true_lbl,
    y_pred=y_pred_lbl,
    raw=y_raw,
    pred_norm=y_pred_lbl,
    n=15,
    seed=42
)
hc_lora

Human check | n=15 | sample accuracy = 53.33%


,idx,correct,gold,pred,raw,pred_norm,text
2,25,False,Office Notes,Neurology,Neurology. SOAP / Chart,Neurology,"DIAGNOSES:,1. Juvenile myoclonic epilepsy.,2. Recent generalized tonic-clonic seizure.,MEDICATIONS:,1. Lamictal 250 mg b.i.d.,2. Depo-Provera.,INTERIM HISTORY: , The patient returns for follow..."
14,89,False,Consult - History and Phy.,UNKNOWN,Consult - History and Phy.,UNKNOWN,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
1,114,False,Rheumatology,General Medicine,General Medicine.Human Resources,General Medicine,"HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria. A was worked up for collagen vascular disease..."
7,142,False,Consult - History and Phy.,Psychiatry / Psychology,Psychiatry / Psychology. SOAP /,Psychiatry / Psychology,"REASON FOR CONSULT:, Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land..."
0,654,False,Gastroenterology,Surgery,Surgery \nThis medical note describes,Surgery,"The patient's abdomen was prepped and draped in the usual sterile fashion. A subumbilical skin incision was made. The Veress needle was inserted, and the patient's abdominal cavity was insufflat..."
3,759,False,Surgery,Pain Management,Pain Management SOAP / Chart /,Pain Management,"PREOPERATIVE DIAGNOSIS:, Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1. Stellate ganglion RFTC (radiofrequency thermocoagulation) left side.,2. Interpret..."
12,913,False,ENT - Otolaryngology,Hematology - Oncology,Hematology - Oncology SOAP,Hematology - Oncology,"PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA: ,Gene..."
9,104,True,Orthopedic,Orthopedic,OrthopedicHuman Resources,Orthopedic,"PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1. Soft tissue mass, right knee.,2. Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio..."
6,228,True,Neurology,Neurology,Neurology.Human Resources,Neurology,"CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig..."
5,250,True,Neurology,Neurology,Neurology. SOAP / Chart,Neurology,"REASON FOR VISIT: ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:, The pat..."


Human check | n=15 | sample accuracy = 53.33%

<div id="df-5dfd411e-ec83-4bd1-b781-14372fb27cd4" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold</th>
      <th>pred</th>
      <th>raw</th>
      <th>pred_norm</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2</th>
      <td>25</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Neurology</td>
      <td>Neurology. SOAP / Chart</td>
      <td>Neurology</td>
      <td>DIAGNOSES:,1.  Juvenile myoclonic epilepsy.,2.  Recent generalized tonic-clonic seizure.,MEDICATIONS:,1.  Lamictal 250 mg b.i.d.,2.  Depo-Provera.,INTERIM HISTORY: , The patient returns for follow...</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>UNKNOWN</td>
      <td>Consult - History and Phy.</td>
      <td>UNKNOWN</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>114</td>
      <td>False</td>
      <td>Rheumatology</td>
      <td>General Medicine</td>
      <td>General Medicine.Human Resources</td>
      <td>General Medicine</td>
      <td>HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria.  A was worked up for collagen vascular disease...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Psychiatry / Psychology</td>
      <td>Psychiatry / Psychology. SOAP /</td>
      <td>Psychiatry / Psychology</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>Surgery</td>
      <td>Surgery \nThis medical note describes</td>
      <td>Surgery</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>759</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Pain Management</td>
      <td>Pain Management SOAP / Chart /</td>
      <td>Pain Management</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1.  Stellate ganglion RFTC (radiofrequency thermocoagulation)  left side.,2.  Interpret...</td>
    </tr>
    <tr>
      <th>12</th>
      <td>913</td>
      <td>False</td>
      <td>ENT - Otolaryngology</td>
      <td>Hematology - Oncology</td>
      <td>Hematology - Oncology SOAP</td>
      <td>Hematology - Oncology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA:  ,Gene...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>104</td>
      <td>True</td>
      <td>Orthopedic</td>
      <td>Orthopedic</td>
      <td>OrthopedicHuman Resources</td>
      <td>Orthopedic</td>
      <td>PREOPERATIVE DIAGNOSIS:, Soft tissue mass, right knee.,POSTOPERATIVE DIAGNOSES:,1.  Soft tissue mass, right knee.,2.  Osteophyte lateral femoral condyle, right knee.,PROCEDURES PERFORMED:, Excisio...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>228</td>
      <td>True</td>
      <td>Neurology</td>
      <td>Neurology</td>
      <td>Neurology.Human Resources</td>
      <td>Neurology</td>
      <td>CC: ,Vertigo.,HX: ,This 61y/o RHF experienced a 2-3 minute episode of lightheadedness while driving home from the dentist in 5/92. In 11/92, while eating breakfast, she suddenly experienced vertig...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>250</td>
      <td>True</td>
      <td>Neurology</td>
      <td>Neurology</td>
      <td>Neurology. SOAP / Chart</td>
      <td>Neurology</td>
      <td>REASON FOR VISIT:  ,This is an 83-year-old woman referred for diagnostic lumbar puncture for possible malignancy by Dr. X. She is accompanied by her daughter.,HISTORY OF PRESENT ILLNESS:,  The pat...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>281</td>
      <td>True</td>
      <td>General Medicine</td>
      <td>General Medicine</td>
      <td>General Medicine. SOAP / Chart</td>
      <td>General Medicine</td>
      <td>CHIEF COMPLAINT:  ,The patient does not have any chief complaint.,HISTORY OF PRESENT ILLNESS:,  This is a 93-year-old female who called up her next-door neighbor to say that she was not feeling we...</td>
    </tr>
    <tr>
      <th>13</th>
      <td>558</td>
      <td>True</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>Cardiovascular / Pulmonary SOAP</td>
      <td>Cardiovascular / Pulmonary</td>
      <td>PREOPERATIVE DIAGNOSIS: , Right hemothorax.,POSTOPERATIVE DIAGNOSIS: , Right hemothorax.,PROCEDURE PERFORMED: , Insertion of a #32 French chest tube on the right hemithorax.,ANESTHESIA: , 1% Lidoc...</td>
    </tr>
    <tr>
      <th>10</th>
      <td>692</td>
      <td>True</td>
      <td>Neurosurgery</td>
      <td>Neurosurgery</td>
      <td>Neurosurgery. Pain Management</td>
      <td>Neurosurgery</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Rule out temporal arteritis.,POSTOPERATIVE DIAGNOSIS:  ,Rule out temporal arteritis.,PROCEDURE:,  Bilateral temporal artery biopsy.,ANESTHESIA:,  Local anesthesia 1% Xylo...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>754</td>
      <td>True</td>
      <td>Surgery</td>
      <td>Surgery</td>
      <td>Surgery.Human Resources (</td>
      <td>Surgery</td>
      <td>PREOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,POSTOPERATIVE DIAGNOSIS: , Secondary capsular membrane, right eye.,PROCEDURE PERFORMED: , YAG laser capsulotomy, right eye.,INDICA...</td>
    </tr>
    <tr>
      <th>11</th>
      <td>758</td>
      <td>True</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>SOAP / Chart / Progress Notes</td>
      <td>SUBJECTIVE:,  He is a 24-year-old male who said that he had gotten into some poison ivy this weekend while he was fishing.  He has had several cases of this in the past and he says that is usually...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-5dfd411e-ec83-4bd1-b781-14372fb27cd4')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>

  </div>


  <div id="id_941e845f-0ba3-4877-9a78-3eb9de6cf586">
    <style>
      .colab-df-generate {
        background-color: #E8F0FE;
        border: none;
        border-radius: 50%;
        cursor: pointer;
        display: none;
        fill: #1967D2;
        height: 32px;
        padding: 0 0 0 0;
        width: 32px;
      }

      .colab-df-generate:hover {
        background-color: #E2EBFA;
        box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
        fill: #174EA6;
      }

      [theme=dark] .colab-df-generate {
        background-color: #3B4455;
        fill: #D2E3FC;
      }

      [theme=dark] .colab-df-generate:hover {
        background-color: #434B5C;
        box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
        filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
        fill: #FFFFFF;
      }
    </style>
    <button class="colab-df-generate" onclick="generateWithVariable('hc_lora')"
            title="Generate code using this dataframe."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px"viewBox="0 0 24 24"
       width="24px">
    <path d="M7,19H8.4L18.45,9,17,7.55,7,17.6ZM5,21V16.75L18.45,3.32a2,2,0,0,1,2.83,0l1.4,1.43a1.91,1.91,0,0,1,.58,1.4,1.91,1.91,0,0,1-.58,1.4L9.25,21ZM18.45,9,17,7.55Zm-12,3A5.31,5.31,0,0,0,4.9,8.1,5.31,5.31,0,0,0,1,6.5,5.31,5.31,0,0,0,4.9,4.9,5.31,5.31,0,0,0,6.5,1,5.31,5.31,0,0,0,8.1,4.9,5.31,5.31,0,0,0,12,6.5,5.46,5.46,0,0,0,6.5,12Z"/>
  </svg>
    </button>
    <script>
      (() => {
      const buttonEl =
        document.querySelector('#id_941e845f-0ba3-4877-9a78-3eb9de6cf586 button.colab-df-generate');
      buttonEl.style.display =
        google.colab.kernel.accessAllowed ? 'block' : 'none';

      buttonEl.onclick = () => {
        google.colab.notebook.generateWithVariable('hc_lora');
      }
      })();
    </script>
  </div>

    </div>
  </div>

In [77]:

hc_lora[hc_lora["correct"] == False].head(10)

,idx,correct,gold,pred,raw,pred_norm,text
2,25,False,Office Notes,Neurology,Neurology. SOAP / Chart,Neurology,"DIAGNOSES:,1. Juvenile myoclonic epilepsy.,2. Recent generalized tonic-clonic seizure.,MEDICATIONS:,1. Lamictal 250 mg b.i.d.,2. Depo-Provera.,INTERIM HISTORY: , The patient returns for follow..."
14,89,False,Consult - History and Phy.,UNKNOWN,Consult - History and Phy.,UNKNOWN,"REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS: ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no..."
1,114,False,Rheumatology,General Medicine,General Medicine.Human Resources,General Medicine,"HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria. A was worked up for collagen vascular disease..."
7,142,False,Consult - History and Phy.,Psychiatry / Psychology,Psychiatry / Psychology. SOAP /,Psychiatry / Psychology,"REASON FOR CONSULT:, Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land..."
0,654,False,Gastroenterology,Surgery,Surgery \nThis medical note describes,Surgery,"The patient's abdomen was prepped and draped in the usual sterile fashion. A subumbilical skin incision was made. The Veress needle was inserted, and the patient's abdominal cavity was insufflat..."
3,759,False,Surgery,Pain Management,Pain Management SOAP / Chart /,Pain Management,"PREOPERATIVE DIAGNOSIS:, Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1. Stellate ganglion RFTC (radiofrequency thermocoagulation) left side.,2. Interpret..."
12,913,False,ENT - Otolaryngology,Hematology - Oncology,Hematology - Oncology SOAP,Hematology - Oncology,"PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA: ,Gene..."


<div id="df-7aa01f6b-e65f-499e-8bc8-c0e5e7cd86b7" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>idx</th>
      <th>correct</th>
      <th>gold</th>
      <th>pred</th>
      <th>raw</th>
      <th>pred_norm</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2</th>
      <td>25</td>
      <td>False</td>
      <td>Office Notes</td>
      <td>Neurology</td>
      <td>Neurology. SOAP / Chart</td>
      <td>Neurology</td>
      <td>DIAGNOSES:,1.  Juvenile myoclonic epilepsy.,2.  Recent generalized tonic-clonic seizure.,MEDICATIONS:,1.  Lamictal 250 mg b.i.d.,2.  Depo-Provera.,INTERIM HISTORY: , The patient returns for follow...</td>
    </tr>
    <tr>
      <th>14</th>
      <td>89</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>UNKNOWN</td>
      <td>Consult - History and Phy.</td>
      <td>UNKNOWN</td>
      <td>REASON FOR CONSULTATION: , Azotemia.,HISTORY OF PRESENT ILLNESS:  ,The patient is a 36-year-old gentleman admitted to the hospital because he passed out at home.,Over the past week, he has been no...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>114</td>
      <td>False</td>
      <td>Rheumatology</td>
      <td>General Medicine</td>
      <td>General Medicine.Human Resources</td>
      <td>General Medicine</td>
      <td>HISTORY: , A is a young lady, who came here with a diagnosis of seizure disorder and history of Henoch-Schonlein purpura with persistent proteinuria.  A was worked up for collagen vascular disease...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>142</td>
      <td>False</td>
      <td>Consult - History and Phy.</td>
      <td>Psychiatry / Psychology</td>
      <td>Psychiatry / Psychology. SOAP /</td>
      <td>Psychiatry / Psychology</td>
      <td>REASON FOR CONSULT:,  Evaluation of alcohol withdrawal and dependance as well as evaluation of anxiety.,HISTORY OF PRESENT ILLNESS: , This is a 50-year-old male who was transferred from Sugar Land...</td>
    </tr>
    <tr>
      <th>0</th>
      <td>654</td>
      <td>False</td>
      <td>Gastroenterology</td>
      <td>Surgery</td>
      <td>Surgery \nThis medical note describes</td>
      <td>Surgery</td>
      <td>The patient's abdomen was prepped and draped in the usual sterile fashion.  A subumbilical skin incision was made.  The Veress needle was inserted, and the patient's abdominal cavity was insufflat...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>759</td>
      <td>False</td>
      <td>Surgery</td>
      <td>Pain Management</td>
      <td>Pain Management SOAP / Chart /</td>
      <td>Pain Management</td>
      <td>PREOPERATIVE DIAGNOSIS:,  Complex Regional Pain Syndrome Type I.,POSTOPERATIVE DIAGNOSIS: , Same.,PROCEDURE:,1.  Stellate ganglion RFTC (radiofrequency thermocoagulation)  left side.,2.  Interpret...</td>
    </tr>
    <tr>
      <th>12</th>
      <td>913</td>
      <td>False</td>
      <td>ENT - Otolaryngology</td>
      <td>Hematology - Oncology</td>
      <td>Hematology - Oncology SOAP</td>
      <td>Hematology - Oncology</td>
      <td>PREOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,POSTOPERATIVE DIAGNOSIS: , Metastatic papillary cancer, left neck.,OPERATION PERFORMED: , Left neck dissection.,ANESTHESIA:  ,Gene...</td>
    </tr>
  </tbody>
</table>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-7aa01f6b-e65f-499e-8bc8-c0e5e7cd86b7')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>

  <style>
    .colab-df-container {
      display:flex;
      gap: 12px;
    }

    .colab-df-convert {
      background-color: #E8F0FE;
      border: none;
      border-radius: 50%;
      cursor: pointer;
      display: none;
      fill: #1967D2;
      height: 32px;
      padding: 0 0 0 0;
      width: 32px;
    }

    .colab-df-convert:hover {
      background-color: #E2EBFA;
      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);
      fill: #174EA6;
    }

    .colab-df-buttons div {
      margin-bottom: 4px;
    }

    [theme=dark] .colab-df-convert {
      background-color: #3B4455;
      fill: #D2E3FC;
    }

    [theme=dark] .colab-df-convert:hover {
      background-color: #434B5C;
      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);
      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));
      fill: #FFFFFF;
    }
  </style>

  </div>


    </div>
  </div>

### Conclusion

Finally, we evaluate a supervised decoder-based approach using LoRA. LoRA keeps the base Qwen2.5-0.5B-Instruct model frozen and trains a small number of additional parameters injected into the attention projection layers. We fine-tune the model to generate exactly one label given a fixed prompt containing the full label list and the clinical note.\
This approach achieves 0.312 accuracy and a **macro F1-score of 0.21**, outperforming both zero-shot and few-shot prompting. Manual inspection shows that the model more often outputs valid labels, but sometimes produces additional trailing text (e.g., partial extra labels or short explanations) that requires normalization before evaluation.

# 8. Final Results and Model Selection

The table below summarizes the performance of the baseline (TF-IDF and log reg), encoder-based models and decoder-based models using macro-averaged F1 as the primary evaluation metric.

| Model | Type | Training | Macro-F1 | Accuracy |
|------|------|----------|----------|----------|
| TF-IDF + Logistic Regression | Classical | Supervised | **0.37** | **0.33**|
| Frozen MedBERT+ classifier head|Encoder | Supervised| 0.02| 0.26|
|MedBERT+ classifier head| Encoder| Supervised | 0.15| 0.27|
| Qwen-0.5B | Decoder | Zero-shot | 0.11 | 0.13|
| Qwen-0.5B | Decoder | Five-shot | 0.01 | 0.03|
| Qwen-0.5B + LoRA | Decoder | PEFT SFT | 0.21 |  0.31|

### Baseline

Baseline, using logistic regression on TF-IDF , gave us the best Macro-F1 value.
This method has several strenghts that explains such a result:
- Need a smaller sample size to be trained (less parameters to be optimized)
- Do not have a maximum token size. We do not need to truncate our medical reports.
- With the weight=balance option, we adapt the weights according to the label frequency, penalizing more errors on less frequent labels.

### Encoder

Structurally, our Med-Bert encoder has three weaknesses, that were not accounted for in this homework:

- We did not take into account that classes are unbalanced in the loss function(i.e. weights are unweighted).
- Medical records are truncated once we reach the max_token size of 512. We loose a lot of informaiton.
- These models would be better with more samples to be trained and more GPU powers for their training hyperparameters (higher batch size, higher number of epochs, smaller learning rates). Neither were available to us.

#### Medbert with frozen weights

The frozen encoder model achieves a very low macro F1. This is expected because the encoder representations are not adapted to our specific 40 class document classification task. With the encoder frozen, only a small classification head is trained, which limits the model to separating classes using fixed pretrained features.

In addition, our medical notes are long and we truncate to 512 tokens, so important information may be removed. Finally, the label distribution is imbalanced, and without explicit class weighting in the training loss, the head can overpredict frequent classes. Together, these factors explain why a frozen encoder performs poorly on macro averaged F1.

### Fine-tuned Medbert:

Allowing the encoder part to be fine-tuned to our specific task and data increased the accuracy by almost a factor of 8 !

However, the macro-f1 score is still lower than our baseline model. This can be explained by the general limitation of our decoder and our setting explained above.

### Decoder

None of the evaluated decoder-based approaches outperform the TF-IDF + Logistic Regression baseline under the primary metric (macro-F1). Zero-shot and five-shot prompting perform poorly on this multi-class classification task, with five-shot prompting even degrading performance due to unstable label usage and limited in-context learning capacity of the small decoder model.

Supervised fine-tuning with LoRA substantially improves decoder performance compared to prompting-based approaches, reaching a macro-F1 of approximately 0.21. Nevertheless, this result still remains below the TF-IDF + Logistic Regression baseline, and thus no decoder-based model is selected as outperforming the baseline under the defined primary metric.

Overall, these findings highlight the strong competitiveness of classical feature-based methods for multi-class medical text classification, especially in settings with limited labeled data and many target classes. They also illustrate the challenges decoder-only language models face when required to reliably generate one label from a large fixed label set.

Limitations and possible improvements:
Performance of the decoder model could likely be improved with additional experimentation, for example:

- using a decoder model better suited to the medical domain (e.g., domain-adapted pretraining),
- exploring larger decoder models with higher representational capacity, which may better support long-context processing and stable label generation,
- training on longer input contexts with less aggressive truncation (medical notes were truncated to approximately 3000 characters for zero- and five-shot prompting and to 1000 characters for LoRA fine-tuning due to computational constraints),
- more careful tuning of LoRA/QLoRA hyperparameters such as learning rate, number of training epochs, rank, and dropout,
- applying constrained decoding or label-aware output mechanisms to reduce generation variability.